In [ ]:
!pip install autocorrect
!pip install --user flair 
!pip install torch


In [ ]:
import torch
import flair
import numpy as np
import pandas as pd
import csv as csv
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer 
from nltk import pos_tag
from nltk import bigrams
from nltk import trigrams
import re, string, unicodedata
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer   
import AddFunctionalities as adjF
import re
from nltk import RegexpParser
from autocorrect import Speller
spell = Speller(lang='en')
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag, word_tokenize
from flair.data import Sentence
from flair.models import SequenceTagger
tagger = SequenceTagger.load('pos-multi')
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()

In [ ]:
 nltk.download('stopwords')

In [ ]:
def treebankTag(text):
    words = nltk.word_tokenize(text)
    return nltk.pos_tag(words)
    

In [ ]:
'''stop word definition'''

stopwords_other = ['a','I','my','you','them','it','its','your','their','they','which','who','mine','not','can','could','will','would','may','might','must','should']
my_stopwords = stopwords.words('English')+stopwords_other


In [ ]:
def chkIfAlreadyExistInArray(word,fullphrase):

    wExist= "false"
    strW = word.split("|")
    strWId = strW[0]
    for i in range(len(fullphrase)):
        listCurr = fullphrase[i]
        listCurr1  =  listCurr.split("|")   
        listCurrId =listCurr1[0]
        if (listCurrId==strWId):
            for b in range (len(strW)):
                strWW = strW[b]
                wExist= "false"
                for c in range (len(listCurr1)):
                    listWW  =listCurr1[c]
                    if (c>0):
                        if ( strWW in listWW):
                            wExist ="true"
                            break
    return wExist;
        

In [ ]:
def chkIfAlreadyExistInArray2(word,fullphrase):

    wExist= "false"
    fullphrase= fullphrase.split("|")
    for i in range(len(fullphrase)):
        listCurr = fullphrase[i]
        wExist= "false"
        if ( word in listCurr):
            wExist ="true"
            break
    return wExist;
        

In [ ]:
def charposition(string, char):
    pos = [] #list to store positions for each 'char' in 'string'
    for n in range(len(string)):
        if string[n] == char:
            pos.append(n)
    return pos



In [ ]:
def extract_fet(sent,rule):
    import re
    grammar = r"R1: {"+rule+"}"
    chunker = nltk.RegexpParser(grammar)
    ne = set()
    chunk = chunker.parse(nltk.pos_tag(nltk.word_tokenize(sent)))
    for tree in chunk.subtrees(filter=lambda t: t.label() == 'R1'):
        ne.add(' '.join([child[0] for child in tree.leaves()]))
    return ne

In [ ]:
ruleList=[]
with open("RulesJan.txt", "r") as file:
    for line in file:
        s = line.rstrip()  
        s.strip('"')
        if (len(s)>2):
            s= s.split(":")
            t= s[1]
            t=t.lstrip('\"')
            ruleList.append([s[0],t,s[2]])
        
file.close()


In [ ]:
# read freq nouns

def chkFreqNoun(word) :
    freqNounList=[]
    with open("NounSetsValuefreq.txt", "r") as file:
        for line in file:
            s = line.rstrip()  
            s.strip('"')
            if (len(s)>2):
                s= s.split("|")
                nT =s[0]
                t= s[1]
                tInt = int(t)
                if (tInt>1):
                    freqNounList.append(nT)
    file.close()
    exist = ""
    op =""
    for a in range ( len (freqNounList)):
        op = freqNounList[a]
        if ( op== word):
            exist ="true"
            break
        else:
            exist ="false"   
    return "true";
    
   

In [ ]:


def chkSentiStrength(word):
    strengthV="false"
    sid = SentimentIntensityAnalyzer()
    a = word
    score =sid.polarity_scores(a)
    compoundV= score['compound']
    if ((compoundV>0.4) or (compoundV<-0.4)): 
        strengthV = "true"

    return strengthV;


In [ ]:
cleanedRS =[]
f=open("NokiaCleanReviews_246.txt", "r")
content = f.read()


xx = content.split("\n")

for i in range (len(xx)):
    if ( len(xx[i])>1):
        cleanedRS.append(xx[i])
        
nounPhrases =[]
sentPhrases=[]
spr1phrases=[]
for u in range(len(cleanedRS)):
    fullPhrase = ''
    sentPhrase=''
    reviewS=  cleanedRS[u]
    reviewS = reviewS.split("|")
    reviewID = reviewS[0]
    rSent  =reviewS[1]

    #import re
    s = rSent

    import re
    string = s
    tempBr = []
    tempSt =[]
    x = re.split(r'[(]',string)

    for i in x:
        if ')' not in i:
            tempSt.append(s)
        else:
            t = re.split(r'[)]',i)
         
            tempBr.append(t[0])
            tempSt.append(s)
    Sent  = re.sub(r" ?\([^)]+\)", "", s)
  
    Sent =Sent.split("*")
    
    for u in range (len(Sent)):
        
        aspectStr1=""
        sentWord=""
        rule=""
        reviewSentence =Sent[u]
        reviewSentenceT = reviewSentence.split()
        revToken=[]
        wdS1= reviewSentence
        wd1 =treebankTag(wdS1)
    
        for h in range (len(wd1)):
            words = wd1[h]
            wordA = words[0]
            wordB = words[1]
            
            if (wordB.startswith('A')):#rule 1
                nextArr = h+1
                if (nextArr< (len(wd1))):
                    nextW =wd1[nextArr]
                    nextwordA = nextW[0]
                    nextwordB = nextW[1]
                    if((nextwordB.startswith('J')) and (adjF.chkIfOpinionWord(nextwordA)=="true")): 
                        nextArr2 = h+2
                        if (nextArr2< (len(wd1))):
                            nextW2 =wd1[nextArr2]
                            next2wordA = nextW2[0]
                            next2wordB = nextW2[1]
                            if (next2wordB.startswith('N')):
                                sentWord = nextwordA+' '+next2wordA
                                rule =" R1 "
                                aspectStr1 = rule+"#"+next2wordA
            if (wordB.startswith('N') and (adjF.chkIfOpinionWord(wordA)=="true")):# Rule 2
                nextArr = h+1
                if (nextArr< (len(wd1))):
                    nextW =wd1[nextArr]
                    nextwordA = nextW[0]
                    nextwordB = nextW[1]
                    if (((nextwordB.startswith('N')))) :
                        rule =" R2a "
                        aspectStr1 = rule+"#"+nextwordA
                        sentWord = wordA+' '+nextwordA
                        nextArr2 = h+2
                        if (nextArr2<(len(wd1))):
                            nextW2 =wd1[nextArr2]
                            next2wordA = nextW2[0]
                            next2wordB = nextW2[1]
                            if ((next2wordB.startswith('C'))):
                                nextArr3 = h+3
                                if (nextArr3<(len(wd1))):
                                    nextW3 =wd1[nextArr3]
                                    next3wordA = nextW3[0]
                                    next3wordB = nextW3[1]
                                    if ((next3wordB.startswith('N'))):
                                        rule =" R2b "
                                        aspectStr1 = rule+"#"+nextwordA+":"+rule+"#"+next3wordA
                                        sentWord = wordA+" "+nextwordA+" "+next3wordA
                                        nextArr4 = h+4
                                        if (nextArr4<(len(wd1))):
                                            nextW4 =wd1[nextArr4]
                                            next4wordA = nextW4[0]
                                            next4wordB = nextW4[1]
                                            if ((next4wordB.startswith('C'))):
                                                nextArr5 = h+5
                                                if (nextArr5<(len(wd1))):
                                                    nextW5 =wd1[nextArr5]
                                                    next5wordA = nextW5[0]
                                                    next5wordB = nextW5[1]
                                                    if ((next5wordB.startswith('N'))):
                                                        rule =" R2c "
                                                        aspectStr1 = rule+"#"+nextwordA+":"+rule+"#"+next3wordA+":"+rule+"#"+next5wordA
                                                        sentWord = wordA+" "+nextwordA+" "+next3wordA+" "+next5wordA
                                                    else :
                                                        rule =" R2d "
                                                        aspectStr1 = rule+"#"+nextwordA+":"+rule+"#"+next3wordA
                                                        sentWord = wordA+" "+nextwordA+" "+next3wordA
                                                else :
                                                    rule =" R2e "
                                                    aspectStr1 = rule+"#"+nextwordA+":"+rule+"#"+next3wordA
                                                    sentWord = wordA+" "+nextwordA+" "+next3wordA
                                            else :
                                                rule =" R2f "
                                                aspectStr1 = rule+"#"+nextwordA+":"+rule+"#"+next3wordA
                                                sentWord = wordA+" "+nextwordA+" "+next3wordA
                                        else :
                                            rule =" R2g"
                                            aspectStr1 = rule+"#"+nextwordA+":"+rule+"#"+next3wordA
                                            sentWord = wordA+" "+nextwordA+" "+next3wordA
                                    else :
                                        rule =" R2h"
                                        aspectStr1 = rule+"#"+nextwordA
                                        sentWord = wordA+" "+nextwordA
                                else :
                                    rule =" R2j"
                                    aspectStr1 = rule+"#"+nextwordA
                                    sentWord = wordA+" "+nextwordA
                            else :
                                rule =" R2k"
                                aspectStr1 = rule+"#"+nextwordA
                                sentWord = wordA+" "+nextwordA
                        else :
                            rule =" R2m"
                            aspectStr1 = rule+"#"+nextwordA
                            sentWord = wordA+" "+nextwordA
                    elif (((nextwordB.startswith('IN')))) :
                        nextArr2 = h+2
                        if (nextArr2<(len(wd1))):
                            nextW2 =wd1[nextArr2]
                            next2wordA = nextW2[0]
                            next2wordB = nextW2[1]
                            if ((next2wordB.startswith('DT'))):
                                nextArr3 = h+3
                                if (nextArr3<(len(wd1))):
                                    nextW3 =wd1[nextArr3]
                                    next3wordA = nextW3[0]
                                    next3wordB = nextW3[1]
                                    if ((next3wordB.startswith('N'))):
                                        nextArr4 = h+4
                                        if (nextArr4<(len(wd1))):
                                            nextW4 =wd1[nextArr4]
                                            next4wordA = nextW4[0]
                                            next4wordB = nextW4[1]
                                            if ((next4wordB.startswith('IN'))):
                                                nextArr5 = h+5
                                                if (nextArr5<(len(wd1))):
                                                    nextW5 =wd1[nextArr5]
                                                    next5wordA = nextW5[0]
                                                    next5wordB = nextW5[1]
                                                    if ((next5wordB.startswith('N'))):
                                                        rule =" R64b "
                                                        aspectStr1 = rule+"#"+next5wordA
                                                        sentWord = wordA+" "+next5wordA
                                                        nextArr6 = h+6
                                                        if (nextArr6<(len(wd1))):
                                                            nextW6 =wd1[nextArr6]
                                                            next6wordA = nextW6[0]
                                                            next6wordB = nextW6[1]    
                                                            if ((next6wordB.startswith('N'))):
                                                                rule =" R64a "
                                                                aspectStr1 = rule+"#"+next5wordA+" "+next6wordA
                                                                sentWord = wordA+" "+next5wordA+" "+next6wordA
                                                            else :
                                                                rule =" R64c "
                                                                aspectStr1 = rule+"#"+next5wordA
                                                                sentWord = wordA+" "+next5wordA
                                                        else :
                                                            rule =" R64d "
                                                            aspectStr1 = rule+"#"+next5wordA
                                                            sentWord = wordA+" "+next5wordA
                                                    elif ((next5wordB.startswith('DT'))): 
                                                        nextArr6 = h+6
                                                        if (nextArr6<(len(wd1))):
                                                            nextW6 =wd1[nextArr6]
                                                            next6wordA = nextW6[0]
                                                            next6wordB = nextW6[1]   
                                                            if ((next6wordB.startswith('N'))):
                                                                rule =" R64e "
                                                                aspectStr1 = rule+"#"+next6wordA
                                                                sentWord = wordA+" "+next6wordA
                                                                nextArr7 = h+7
                                                                if (nextArr7<(len(wd1))):
                                                                    nextW7 =wd1[nextArr7]
                                                                    next7wordA = nextW7[0]
                                                                    next7wordB = nextW7[1]    
                                                                    if ((next7wordB.startswith('N'))):
                                                                        rule =" R64f "
                                                                        aspectStr1 = rule+"#"+next6wordA+" "+next7wordA
                                                                    else :
                                                                        rule =" R64g "
                                                                        aspectStr1 = rule+"#"+next6wordA
                                                                        sentWord = wordA+" "+next6wordA
                                                                else :
                                                                    rule =" R64h "
                                                                    aspectStr1 = rule+"#"+next6wordA
                                                                    sentWord = wordA+" "+next6wordA
                    elif (((nextwordB.startswith('DT')))) :
                        nextArr2 = h+2
                        if (nextArr2<(len(wd1))):
                            nextW2 =wd1[nextArr2]
                            next2wordA = nextW2[0]
                            next2wordB = nextW2[1]
                            if ((next2wordB.startswith('N'))):
                                rule =" R66a "
                                aspectStr1 = rule+"#"+next2wordA
                                sentWord = wordA+" "+next2wordA
                                nextArr3 = h+3
                                if (nextArr3<(len(wd1))):
                                    nextW3 =wd1[nextArr3]
                                    next3wordA = nextW3[0]
                                    next3wordB = nextW3[1]
                                    if ((next3wordB.startswith('N'))):
                                        rule =" R66b "
                                        aspectStr1 = rule+"#"+next2wordA +" "+next3wordA
                                        sentWord = wordA+" "+next2wordA+" "+next3wordA
                                    else :
                                        rule =" R66c "
                                        aspectStr1 = rule+"#"+next2wordA
                                        sentWord = wordA+" "+next2wordA
                                        
                                else :
                                        rule =" R66d "
                                        aspectStr1 = rule+"#"+next2wordA        
                                        sentWord = wordA+" "+next2wordA
                      
                        
            if (((wordB.startswith('J'))) and (adjF.chkIfOpinionWord(wordA)=="true")) :#rule3
                nextArr = h+1
                if (nextArr< (len(wd1))):
                    nextW =wd1[nextArr]
                    nextwordA = nextW[0]
                    nextwordB = nextW[1]
                    if (((nextwordB.startswith('N')))) :
                        rule =" R3a "
                        aspectStr1 = rule+"#"+nextwordA
                        sentWord = wordA+" "+nextwordA
                        
                        nextArr2 = h+2
                        if (nextArr2<(len(wd1))):
                            nextW2 =wd1[nextArr2]
                            next2wordA = nextW2[0]
                            next2wordB = nextW2[1]
                            if ((next2wordB.startswith('C'))):
                                nextArr3 = h+3
                                if (nextArr3<(len(wd1))):
                                    nextW3 =wd1[nextArr3]
                                    next3wordA = nextW3[0]
                                    next3wordB = nextW3[1]
                                    if ((next3wordB.startswith('N'))):
                                        rule =" R3b "
                                        aspectStr1 = rule+"#"+nextwordA+":"+rule+"#"+next3wordA
                                        sentWord = wordA+" "+nextwordA+" "+next3wordA
                                        nextArr4 = h+4
                                        if (nextArr4<(len(wd1))):
                                            nextW4 =wd1[nextArr4]
                                            next4wordA = nextW4[0]
                                            next4wordB = nextW4[1]
                                            if ((next4wordB.startswith('N'))):
                                                rule =" R3c "
                                                aspectStr1 = rule+"#"+nextwordA+":"+rule+"#"+next3wordA+" "+next4wordA
                                                sentWord = wordA+" "+nextwordA+" "+next3wordA+" "+next4wordA
                                            else :
                                                rule =" R3d "
                                                aspectStr1 = rule+"#"+nextwordA+":"+rule+"#"+next3wordA
                                                sentWord = wordA+" "+nextwordA+" "+next3wordA
                                        else :
                                            rule =" R3e "
                                            aspectStr1 = rule+"#"+nextwordA+":"+rule+"#"+next3wordA
                                            sentWord = wordA+" "+nextwordA+" "+next3wordA
                                    else :
                                        rule =" R3f "
                                        aspectStr1 = rule+"#"+nextwordA
                                        sentWord = wordA+" "+nextwordA
                            elif ((next2wordB.startswith('N'))):
                                rule =" R3g "
                                aspectStr1 = rule+"#"+nextwordA+" "+next2wordA
                                sentWord = wordA+" "+nextwordA+" "+next2wordA
                                nextArr3 = h+3
                                if (nextArr3<(len(wd1))):
                                    nextW3 =wd1[nextArr3]
                                    next3wordA = nextW3[0]
                                    next3wordB = nextW3[1]
                                    if ((next3wordB.startswith('N'))):
                                        rule =" R3h "
                                        aspectStr1 = rule+"#"+nextwordA+" "+next2wordA+" "+next3wordA
                                        sentWord = wordA+" "+nextwordA+" "+next2wordA+" "+next3wordA
                                    else :
                                        rule =" R3i "
                                        aspectStr1 = rule+"#"+nextwordA+" "+next2wordA
                                        sentWord = wordA+" "+nextwordA+" "+next2wordA
                                        
                                else :
                                    rule =" R3j "
                                    aspectStr1 = rule+"#"+nextwordA+" "+next2wordA
                                    sentWord = wordA+" "+nextwordA+" "+next2wordA
                                    
                            else:
                                rule =" R3k "
                                sentWord =wordA+" "+nextwordA
                                if (chkSentiStrength(sentWord)=="true"):
                                    aspectStr1 = rule+"#"+nextwordA
                                    sentWord = wordA+" "+nextwordA
                                    
                        else:
                            rule =" R3l "
                            aspectStr1 = rule+"#"+nextwordA
                            sentWord = wordA+" "+nextwordA
                              
                    elif((nextwordB.startswith('R'))):
                        nextArr2 = h+2
                        if (nextArr2<(len(wd1))):
                                    nextW2 =wd1[nextArr2]
                                    next2wordA = nextW2[0]
                                    next2wordB = nextW2[1]
                                    if ((next2wordB.startswith('N'))): #Rule 5
                                        rule =" R5a "
                                        aspectStr1 = rule+"#"+next2wordA
                                        sentWord = wordA+" "+next2wordA
                                        nextArr3 = h+3
                                        if (nextArr3<(len(wd1))):
                                            nextW3 =wd1[nextArr3]
                                            next3wordA = nextW3[0]
                                            next3wordB = nextW3[1]
                                            if ((next3wordB.startswith('C'))):
                                                nextArr4 = h+4
                                                if (nextArr4<(len(wd1))):
                                                    nextW4 =wd1[nextArr4]
                                                    next4wordA = nextW4[0]
                                                    next4wordB = nextW4[1]
                                                    if ((next4wordB.startswith('N'))):
                                                        rule =" R5b"
                                                        aspectStr1 = rule+"#"+next2wordA+":"+rule+"#"+next4wordA
                                                        sentWord = wordA+" "+next2wordA+" "+next4wordA
                                                        nextArr5 = h+5
                                                        if (nextArr5<(len(wd1))):
                                                            nextW5 =wd1[nextArr5]
                                                            next5wordA = nextW5[0]
                                                            next5wordB = nextW5[1]
                                                            if ((next5wordB.startswith('N'))):
                                                                rule =" R5c "
                                                                aspectStr1 = rule+"#"+next2wordA+":"+rule+"#"+next4wordA+" "+next5wordA
                                                                sentWord = wordA+" "+next2wordA+" "+next4wordA+" "+next5wordA
                                                            else :
                                                                rule =" R5d "
                                                                aspectStr1 = rule+"#"+next2wordA+":"+rule+"#"+next4wordA
                                                                sentWord = wordA+" "+next2wordA+" "+next4wordA
                                                        else :
                                                            rule =" R5e "
                                                            aspectStr1 = rule+"#"+next2wordA+":"+rule+"#"+next4wordA
                                                            sentWord = wordA+" "+next2wordA+" "+next4wordA
                                                    else :
                                                        rule =" R5f "
                                                        aspectStr1 = rule+"#"+next2wordA
                                                        sentWord = wordA+" "+next2wordA
                                            elif ((next3wordB.startswith('N'))):
                                                rule =" R5g "
                                                aspectStr1 = rule+"#"+next2wordA+" "+next3wordA
                                                sentWord = wordA+" "+next2wordA+" "+next3wordA
                                                nextArr4 = h+4
                                                if (nextArr4<(len(wd1))):
                                                    nextW4 =wd1[nextArr4]
                                                    next4wordA = nextW4[0]
                                                    next4wordB = nextW4[1]
                                                    if ((next4wordB.startswith('N'))):
                                                        rule =" R5h"
                                                        aspectStr1 = rule+"#"+next2wordA+" "+next3wordA+" "+next4wordA
                                                        sentWord = wordA+" "+next2wordA+" "+next3wordA+" "+next4wordA
                                                    else :
                                                        rule =" R5i "
                                                        aspectStr1 = rule+"#"+next2wordA+" "+next3wordA
                                                        sentWord = wordA+" "+next2wordA+" "+next3wordA
                                                else :
                                                    rule =" R5j "
                                                    aspectStr1 = rule+"#"+next2wordA+" "+next3wordA
                                                    sentWord = wordA+" "+next2wordA+" "+next3wordA
                                            else:
                                                rule =" R5k "
                                                aspectStr1 = rule+"#"+next2wordA                  
                                                sentWord = wordA+" "+next2wordA
                                    elif (((next2wordB.startswith('V')))) :
                                        nextArr3 = h+3
                                        if (nextArr3<(len(wd1))):
                                            nextW3 =wd1[nextArr3]
                                            next3wordA = nextW3[0]
                                            next3wordB = nextW3[1]
                                            if ((next3wordB.startswith('N'))):#Rule 6
                                                rule =" R5l "
                                                aspectStr1 = rule+"#"+next3wordA
                                                sentWord = wordA+" "+next3wordA
                                                nextArr4 = h+4
                                                if (nextArr4<(len(wd1))):
                                                    nextW4 =wd1[nextArr4]
                                                    next4wordA = nextW3[0]
                                                    next4wordB = nextW3[1]
                                                    if ((next4wordB.startswith('N'))):
                                                        rule =" R5m "
                                                        aspectStr1 = rule+"#"+next3wordA+" "+next4wordA
                                                        sentWord = wordA+" "+next3wordA+" "+next4wordA
                                                    else :
                                                        rule =" R5n "
                                                        aspectStr1 = rule+"#"+next3wordA
                                                        sentWord = wordA+" "+next3wordA
                                                else :
                                                    rule =" R5p "
                                                    aspectStr1 = rule+"#"+next3wordA
                                                    sentWord = wordA+" "+next3wordA
                    elif (((nextwordB.startswith('J')))) :
                        nextArr2 = h+2
                        if (nextArr2<(len(wd1))):
                            nextW2 =wd1[nextArr2]
                            next2wordA = nextW2[0]
                            next2wordB = nextW2[1]
                            if (not (next2wordB.startswith('N'))): #Rule 7
                                rule =" R7a "
                                aspectStr1 = rule+"#"+next2wordA
                                sentWord = wordA+" "+next2wordA
                    elif((nextwordB.startswith('V'))):
                        nextArr2 = h+2
                        if (nextArr2<(len(wd1))):
                                    nextW2 =wd1[nextArr2]
                                    next2wordA = nextW2[0]
                                    next2wordB = nextW2[1]
                                    if ((next2wordB.startswith('N'))): #Rule 9
                                        rule =" R9a "
                                        aspectStr1 = rule+"#"+next2wordA                      
                                        sentWord = wordA+" "+next2wordA
                                        nextArr3 = h+3
                                        if (nextArr3<(len(wd1))):
                                            nextW3 =wd1[nextArr3]
                                            next3wordA = nextW3[0]
                                            next3wordB = nextW3[1]
                                            if ((next3wordB.startswith('C'))):
                                                nextArr4 = h+4
                                                if (nextArr4<(len(wd1))):
                                                    nextW4 =wd1[nextArr4]
                                                    next4wordA = nextW4[0]
                                                    next4wordB = nextW4[1]
                                                    if ((next4wordB.startswith('N'))):
                                                        rule =" R9b"
                                                        aspectStr1 = rule+"#"+next2wordA+":"+rule+"#"+next4wordA
                                                        sentWord = wordA+" "+next2wordA+" "+next4wordA
                                                        nextArr5 = h+5
                                                        if (nextArr5<(len(wd1))):
                                                            nextW5 =wd1[nextArr5]
                                                            next5wordA = nextW5[0]
                                                            next5wordB = nextW5[1]
                                                            if ((next5wordB.startswith('N'))):
                                                                rule =" R9c"
                                                                aspectStr1 = rule+"#"+next2wordA+":"+rule+"#"+next4wordA+" "+next5wordA
                                                                sentWord = wordA+" "+next2wordA+" "+next4wordA+" "+next5wordA
                                                            else :
                                                                rule =" R9d "
                                                                aspectStr1 = rule+"#"+next2wordA+":"+rule+"#"+next4wordA
                                                                sentWord = wordA+" "+next2wordA+" "+next4wordA
                                                        else :
                                                            rule =" R9e"
                                                            aspectStr1 = rule+"#"+next2wordA+":"+rule+"#"+next4wordA
                                                            sentWord = wordA+" "+next2wordA+" "+next4wordA
                                                    else :
                                                        rule =" R9f"
                                                        aspectStr1 = rule+"#"+next2wordA
                                                        sentWord = wordA+" "+next2wordA
                                            elif ((next3wordB.startswith('N'))):
                                                rule =" R9g"
                                                aspectStr1 = rule+"#"+next2wordA+" "+next3wordA                
                                                sentWord = wordA+" "+next2wordA+" "+next3wordA
                                                nextArr4 = h+4
                                                if (nextArr4<(len(wd1))):
                                                    nextW4 =wd1[nextArr4]
                                                    next4wordA = nextW4[0]
                                                    next4wordB = nextW4[1]
                                                    if ((next4wordB.startswith('N'))):
                                                        rule =" R9h"
                                                        aspectStr1 = rule+"#"+next2wordA+" "+next3wordA+" "+next4wordA
                                                        sentWord = wordA+" "+next2wordA+" "+next3wordA+" "+next4wordA
                                                    else :
                                                        rule =" R9i "
                                                        aspectStr1 = rule+"#"+next2wordA+" "+next3wordA
                                                        sentWord = wordA+" "+next2wordA+" "+next3wordA
                                                else :
                                                    rule =" R9j "
                                                    aspectStr1 = rule+"#"+next2wordA+" "+next3wordA
                                                    sentWord = wordA+" "+next2wordA+" "+next3wordA
                                            else:
                                                rule =" R9k "
                                                aspectStr1 = rule+"#"+next2wordA
                                                sentWord = wordA+" "+next2wordA
                                        else:
                                            rule =" R9l "
                                            aspectStr1 = rule+"#"+next2wordA
                                            sentWord = wordA+" "+next2wordA
                                    if ((next2wordB.startswith('DT'))):
                                            nextArr3 = h+3
                                            if (nextArr3<(len(wd1))):
                                                nextW3 =wd1[nextArr3]
                                                next3wordA = nextW3[0]
                                                next3wordB = nextW3[1]
                                                if ((next3wordB.startswith('N'))):
                                                    rule =" R62a"
                                                    aspectStr1 = rule+"#"+next3wordA
                                                    sentWord = wordA+" "+next3wordA
                                                    nextArr4 = h+4
                                                    if (nextArr4<(len(wd1))):
                                                        nextW4 =wd1[nextArr4]
                                                        next4wordA = nextW4[0]
                                                        next4wordB = nextW4[1]
                                                        if ((next4wordB.startswith('CC'))):
                                                            nextArr5 = h+5
                                                            if (nextArr5<(len(wd1))):
                                                                nextW5 =wd1[nextArr5]
                                                                next5wordA = nextW5[0]
                                                                next5wordB = nextW5[1]
                                                                if ((next5wordB.startswith('DT'))):
                                                                        nextArr6 = h+6
                                                                        if (nextArr6<(len(wd1))):
                                                                            nextW6 =wd1[nextArr6]
                                                                            next6wordA = nextW6[0]
                                                                            next6wordB = nextW6[1]
                                                                            if ((next6wordB.startswith('N'))):
                                                                                rule =" R62b"
                                                                                aspectStr1 = rule+"#"+next3wordA+":"+rule+"#"+next6wordA
                                                                                sentWord = wordA+" "+next3wordA+" "+next6wordA
                                                                                nextArr7 = h+7
                                                                                if (nextArr7<(len(wd1))):
                                                                                    nextW7 =wd1[nextArr7]
                                                                                    next7wordA = nextW7[0]
                                                                                    next7wordB = nextW7[1]
                                                                                    if ((next7wordB.startswith('CC'))):
                                                                                        nextArr8 = h+8
                                                                                        if (nextArr8<(len(wd1))):
                                                                                            nextW8 =wd1[nextArr8]
                                                                                            next8wordA = nextW8[0]
                                                                                            next8wordB = nextW8[1]
                                                                                            if ((next8wordB.startswith('DT'))):
                                                                                                nextArr9 = h+9
                                                                                                if (nextArr9<(len(wd1))):
                                                                                                    nextW9 =wd1[nextArr9]
                                                                                                    next9wordA = nextW9[0]
                                                                                                    next9wordB = nextW9[1]
                                                                                                    if ((next9wordB.startswith('N'))):
                                                                                                        rule =" R62c"
                                                                                                        aspectStr1 = rule+"#"+next3wordA+":"+rule+"#"+next6wordA+":"+rule+"#"+next9wordA
                                                                                                        sentWord = wordA+" "+next3wordA+" "+next6wordA+" "+next9wordA
                                                                                                    else:
                                                                                                        rule =" R62d"
                                                                                                        aspectStr1 = rule+"#"+next3wordA+":"+rule+"#"+next6wordA+":"+rule+"#"+next9wordA
                                                                                                        sentWord = wordA+" "+next3wordA+" "+next6wordA+" "+next9wordA
                                                                                                else:
                                                                                                    rule =" R62e"
                                                                                                    aspectStr1 = rule+"#"+next3wordA+":"+rule+"#"+next6wordA
                                                                                                    sentWord = wordA+" "+next3wordA+" "+next6wordA
                                                                                            elif ((next8wordB.startswith('N'))):
                                                                                                    rule =" R62f"
                                                                                                    aspectStr1 = rule+"#"+next3wordA+":"+rule+"#"+next6wordA+":"+rule+"#"+next8wordA
                                                                                                    sentWord = wordA+" "+next3wordA+" "+next6wordA+" "+nex8wordA
                                                                                            elif ((next8wordB.startswith('V'))):
                                                                                                    rule =" R62g"
                                                                                                    aspectStr1 = rule+"#"+next3wordA+":"+rule+"#"+next6wordA+":"+rule+"#"+next8wordA
                                                                                                    sentWord = wordA+" "+next3wordA+" "+next6wordA+" "+nex8wordA
                                                                                            else:
                                                                                                rule =" R62h"
                                                                                                aspectStr1 = rule+"#"+next3wordA+":"+rule+"#"+next6wordA+":"+rule+"#"+next8wordA
                                                                                                sentWord = wordA+" "+next3wordA+" "+next6wordA+" "+nex8wordA
                                                                                        else:
                                                                                            rule =" R62j"
                                                                                            aspectStr1 = rule+"#"+next3wordA+":"+rule+"#"+next6wordA
                                                                                            sentWord = wordA+" "+next3wordA+" "+next6wordA
                                                                                    else:
                                                                                            rule =" R62k"
                                                                                            aspectStr1 = rule+"#"+next3wordA+":"+rule+"#"+next6wordA
                                                                                            sentWord = wordA+" "+next3wordA+" "+next6wordA
                                                                                else:
                                                                                    rule =" R62m"
                                                                                    aspectStr1 = rule+"#"+next3wordA+rule+"#"":"+next6wordA
                                                                                    sentWord = wordA+" "+next3wordA+" "+next6wordA
                                                                            else:
                                                                                rule =" R62n"
                                                                                aspectStr1 = rule+"#"+next3wordA+":"+rule+"#"+next6wordA
                                                                                sentWord = wordA+" "+next3wordA+" "+next6wordA
                                                                        else:
                                                                            rule =" R62p"
                                                                            aspectStr1 = rule+"#"+next3wordA
                                                                            sentWord = wordA+" "+next3wordA
                                                                else:
                                                                    rule =" R62q"
                                                                    aspectStr1 = rule+"#"+next3wordA
                                                                    sentWord = wordA+" "+next3wordA
                                                            else:
                                                                rule =" R62r"
                                                                aspectStr1 = rule+"#"+next3wordA
                                                                sentWord = wordA+" "+next3wordA
                                                        else:
                                                            rule =" R62s"
                                                            aspectStr1 = rule+"#"+next3wordA
                                                            sentWord = wordA+" "+next3wordA
                                                    else:
                                                        rule =" R62t"
                                                        aspectStr1 = rule+"#"+next3wordA
                                                        sentWord = wordA+" "+next3wordA
                                                else:
                                                    rule =" R62u"
                                                    aspectStr1 = rule+"#"+next3wordA
                                                    sentWord = wordA+" "+next3wordA
            if (((wordB.startswith('J'))) and (adjF.chkIfOpinionWord(wordA)=="false")) :#rule10        
                nextArr = h+1
                if (nextArr< (len(wd1))):
                    nextW =wd1[nextArr]
                    nextwordA = nextW[0]
                    nextwordB = nextW[1]
                    if (((nextwordB.startswith('V')))) :
                        nextArr2 = h+2
                        if (nextArr2<(len(wd1))):
                            nextW2 =wd1[nextArr2]
                            next2wordA = nextW2[0]
                            next2wordB = nextW2[1]
                            if ((next2wordB.startswith('R'))):
                                nextArr3 = h+3
                                if (nextArr3<(len(wd1))):
                                    nextW3 =wd1[nextArr3]
                                    next3wordA = nextW3[0]
                                    next3wordB = nextW3[1]
                                    if ((next3wordB.startswith('J')) and (adjF.chkIfOpinionWord(next3wordA)=="true")) :#rule10 
                                        rule =" R10a"
                                        aspectStr1 = rule+"#"+wordA
                                        sentWord = wordA+" "+nextwordA+" "+next2wordA+" "+next3wordA
                    elif (((nextwordB.startswith('TO')))) :
                        nextArr2 = h+2
                        if (nextArr2<(len(wd1))):
                            nextW2 =wd1[nextArr2]
                            next2wordA = nextW2[0]
                            next2wordB = nextW2[1]
                            if ((next2wordB.startswith('V'))): #Rule 8
                                rule =" R8a "
                                aspectStr1 = rule+"#"+next2wordA
                                sentWord = wordA+" "+nextwordA+" "+next2wordA
            if (((wordB.startswith('J'))) and (adjF.chkIfOpinionWord(wordA)=="true")):
                nextArr = h+1
                if (nextArr< (len(wd1))):
                    nextW =wd1[nextArr]
                    nextwordA = nextW[0]
                    nextwordB = nextW[1]
                                                               
                    if (((nextwordB.startswith('TO')))) :
                        nextArr2 = h+2
                        if (nextArr2<(len(wd1))):
                            nextW2 =wd1[nextArr2]
                            next2wordA = nextW2[0]
                            next2wordB = nextW2[1]
                            if ((next2wordB.startswith('V'))): #Rule 8
                                rule =" R8b "
                                aspectStr1 = rule+"#"+ next2wordA    
                                sentWord = wordA+" "+nextwordA+" "+next2wordA
            if (((wordB.startswith('N'))) and (adjF.chkIfOpinionWord(wordA)=="true")) :#rule11      
                nextArr = h+1
                if (nextArr< (len(wd1))):
                    nextW =wd1[nextArr]
                    nextwordA = nextW[0]
                    nextwordB = nextW[1]
                    if (((nextwordB.startswith('T')))) :
                        nextArr2 = h+2
                        if (nextArr2<(len(wd1))):
                            nextW2 =wd1[nextArr2]
                            next2wordA = nextW2[0]
                            next2wordB = nextW2[1]
                            if ((next2wordB.startswith('N'))): #Rule 9
                                rule =" R11a "
                                aspectStr1 = rule+"#"+ next2wordA       
                                sentWord = wordA+" "+nextwordA+" "+next2wordA
                                nextArr3 = h+3
                                if (nextArr3<(len(wd1))):
                                    nextW3 =wd1[nextArr3]
                                    next3wordA = nextW3[0]
                                    next3wordB = nextW3[1]
                                    if ((next3wordB.startswith('C'))):
                                        nextArr4 = h+4
                                        if (nextArr4<(len(wd1))):
                                            nextW4 =wd1[nextArr4]
                                            next4wordA = nextW4[0]
                                            next4wordB = nextW4[1]
                                            if ((next4wordB.startswith('N'))):
                                                rule =" R11b"
                                                aspectStr1 = rule+"#"+next2wordA+":"+rule+"#"+next4wordA
                                                sentWord = wordA+" "+next2wordA+" "+next4wordA
                                                nextArr5 = h+5
                                                if (nextArr5<(len(wd1))):
                                                    nextW5 =wd1[nextArr5]
                                                    next5wordA = nextW5[0]
                                                    next5wordB = nextW5[1]
                                                    if ((next5wordB.startswith('N'))):
                                                        rule =" R11c "
                                                        aspectStr1 = rule+"#"+next2wordA+":"+rule+"#"+next4wordA+" "+next5wordA
                                                        sentWord = wordA+" "+next2wordA+" "+next4wordA+" "+next5wordA
                                                    else :
                                                        rule =" R11d "
                                                        aspectStr1 = rule+"#"+next2wordA+":"+rule+"#"+next4wordA
                                                        sentWord = wordA+" "+next2wordA+" "+next4wordA
                                                else :
                                                    rule =" R11e "
                                                    aspectStr1 = rule+"#"+next2wordA+":"+rule+"#"+next4wordA
                                                    sentWord = wordA+" "+next2wordA+" "+next4wordA
                                            else :
                                                rule =" R11f "
                                                aspectStr1 = rule+"#"+next2wordA
                                                sentWord = wordA+" "+nextwordA
                                    elif ((next3wordB.startswith('N'))):
                                        rule =" R11g "
                                        aspectStr1 = rule+"#"+next2wordA+" "+next3wordA
                                        sentWord = wordA+" "+next2wordA+" "+next3wordA
                                        nextArr4 = h+4
                                        if (nextArr4<(len(wd1))):
                                            nextW4 =wd1[nextArr4]
                                            next4wordA = nextW4[0]
                                            next4wordB = nextW4[1]
                                            if ((next4wordB.startswith('N'))):
                                                rule =" R11h "
                                                aspectStr1 = rule+"#"+next2wordA+" "+next3wordA+" "+next4wordA
                                                sentWord = wordA+" "+next2wordA+" "+next3wordA+" "+next4wordA
                                            else :
                                                rule =" R11j "
                                                aspectStr1 = rule+"#"+next2wordA+" "+next3wordA
                                                sentWord = wordA+" "+next2wordA+" "+next3wordA
                                        else :
                                            rule =" R11k "
                                            aspectStr1 = rule+"#"+next2wordA+" "+next3wordA
                                            sentWord = wordA+" "+next2wordA+" "+next3wordA
                                    else:
                                            rule =" R11m "
                                            aspectStr1 = rule+"#"+next2wordA
                                            sentWord = wordA+" "+next2wordA
                                else:
                                    rule =" R11n "
                                    aspectStr1 = rule+"#"+next2wordA
                                    sentWord = wordA+" "+next2wordA
                    if (((nextwordB.startswith('IN')))) :
                        nextArr2 = h+2
                        if (nextArr2<(len(wd1))):
                            nextW2 =wd1[nextArr2]
                            next2wordA = nextW2[0]
                            next2wordB = nextW2[1]
                            if ((next2wordB.startswith('JJ'))): #Rule73
                                rule =" R73a "
                                aspectStr1 = rule+"#"+next2wordA
                                sentWord = wordA+" "+next2wordA
            if ((wordB.startswith('N')) and (adjF.chkIfOpinionWord(wordA)=="false")) :#rule12      
                nextArr = h+1
                if (nextArr< (len(wd1))):
                    nextW =wd1[nextArr]
                    nextwordA = nextW[0]
                    nextwordB = nextW[1]
                    if (((nextwordB.startswith('R')) and (adjF.chkIfOpinionWord(nextwordA)=="true"))):#rule16 :
                        rule =" R67a"
                        aspectStr1 = rule+"#"+wordA
                        sentWord = wordA+" "+nextwordA
                        nextArr2 = h+2
                        if (nextArr2<(len(wd1))):
                            nextW2 =wd1[nextArr2]
                            next2wordA = nextW2[0]
                            next2wordB = nextW2[1]
                              
                            if ((next2wordB.startswith('N'))):#rule12  
                                    rule =" R16a "
                                    aspectStr1 = rule+"#"+wordA   
                                    sentWord = wordA+" "+nextwordA+" "+next2wordA
                                    if ((prevArr >= 0)and (prevArr< (len(wd1)))):
                                        prevW =wd1[prevArr]
                                        prevWordA = prevW[0]
                                        prevWordB = prevW[1]
                                        if ((prevWordB.startswith('N'))):
                                            rule =" R16b "
                                            aspectStr1 = rule+"#"+prevWordA+" "+wordA
                                            sentWord = prevWordA+" "+wordA+" "+nextwordA
                                        else:
                                            rule =" R16c "
                                            aspectStr1 = rule+"#"+wordA
                                            sentWord = wordA+" "+nextwordA
                                    else:
                                        rule =" R16d "
                                        aspectStr1 = rule+"#"+wordA
                                        sentWord = wordA+" "+nextwordA
                            elif ((next2wordB.startswith('N')or (next2wordB.startswith('V'))) and (adjF.chkIfOpinionWord(next2wordA)=="true")):#rule12  
                                    rule =" R12a "
                                    aspectStr1 = rule+"#"+wordA
                                    sentWord = wordA+" "+nextwordA+" "+next2wordB
                                    prevArr = h-1
                                    if ((prevArr >= 0)and (prevArr< (len(wd1)))):
                                        prevW =wd1[prevArr]
                                        prevWordA = prevW[0]
                                        prevWordB = prevW[1]
                                        if ((prevWordB.startswith('N'))):
                                            rule =" R12b "
                                            aspectStr1 = rule+"#"+prevWordA+" "+wordA
                                            sentWord = prevWordA+" "+wordA+" "+next2wordA
                                    else:
                                            rule =" R12d "
                                            aspectStr1 = rule+"#"+wordA
                                            sentWord = wordA+" "+next2wordA
                            elif ((next2wordB.startswith('V'))): #Rule 13
                                nextArr3 = h+3
                                if (nextArr3<(len(wd1))):
                                    nextW3 =wd1[nextArr3]
                                    next3wordA = nextW3[0]
                                    next3wordB = nextW3[1]
                                    if ((next3wordB.startswith('N')) and (adjF.chkIfOpinionWord(next3wordA)=="true")) :#rule13  
                                        rule =" R13a "
                                        aspectStr1 = rule+"#"+wordA
                                        sentWord = wordA+" "+next3wordA
                                        prevArr = h-1
                                        if ((prevArr >= 0)and (prevArr< (len(wd1)))):
                                            prevW =wd1[prevArr]
                                            prevWordA = prevW[0]
                                            prevWordB = prevW[1]
                                            if ((prevWordB.startswith('N'))):
                                                rule =" R13b "
                                                aspectStr1 = rule+"#"+prevWordA+" "+wordA
                                                sentWord = prevWordA+" "+wordA+" "+next3wordA
                                            else:
                                                rule =" R13c "
                                                aspectStr1 = rule+"#"+wordA
                                                sentWord = prevWordA+" "+wordA+" "+next3wordA
                                        else :
                                            rule =" R13d "
                                            aspectStr1 = rule+"#"+wordA
                                            sentWord = wordA+" "+next3wordA
                            elif (((next2wordB.startswith('J')) or  (next2wordB.startswith('V'))) and (adjF.chkIfOpinionWord(next2wordA)=="true")):#rule23
                                rule =" R28a "
                                aspectStr1 = rule+"#"+wordA 
                                sentWord =wordA+" "+next2wordA
                                prevArr = h-1
                                if ((prevArr >= 0)and (prevArr< (len(wd1)))):
                                    prevW =wd1[prevArr]
                                    prevWordA = prevW[0]
                                    prevWordB = prevW[1]
                                    if ((prevWordB.startswith('N'))):
                                        rule =" R28b "
                                        aspectStr1 = rule+"#"+prevWordA+" "+wordA
                                        sentWord =prevWordA+" "+wordA+" "+next2wordA
                                    else:
                                        rule =" R28c "
                                        aspectStr1 = rule+"#"+wordA
                                        sentWord =wordA+" "+next2wordA
                                else :
                                    rule =" R28d "
                                    aspectStr1 = rule+"#"+wordA  
                                    sentWord =wordA+" "+next2wordA
                    elif ((nextwordB.startswith('V'))): #Rule 13
                                nextArr2 = h+2
                                if (nextArr2<(len(wd1))):
                                    nextW2 =wd1[nextArr2]
                                    next2wordA = nextW2[0]
                                    next2wordB = nextW2[1]
                                    if ((next2wordB.startswith('R'))) :#rule13  
                                        
                                        if ((adjF.chkIfOpinionWord(next2wordA)=="true")):
                                            rule =" R76a "
                                            aspectStr1 = rule+"#"+wordA
                                            sentWord =wordA+" "+next2wordA
                                        nextArr3 = h+3
                                        if (nextArr3<(len(wd1))):
                                            nextW3 =wd1[nextArr3]
                                            next3wordA = nextW3[0]
                                            next3wordB = nextW3[1]
                                            if ((next3wordB.startswith('N')) and (adjF.chkIfOpinionWord(next3wordA)=="true")):#rule13 
                                                rule =" R13g "
                                                aspectStr1 = rule+"#"+wordA
                                                sentWord =wordA+" "+next3wordA
                                                prevArr = h-1
                                                if ((prevArr >= 0)and (prevArr< (len(wd1)))):
                                                    prevW =wd1[prevArr]
                                                    prevWordA = prevW[0]
                                                    prevWordB = prevW[1]
                                                else:
                                                    rule =" R13k "
                                                    aspectStr1 = rule+"#"+wordA
                                                    sentWord =wordA+" "+next3wordA
                                            elif ((next3wordB.startswith('J')) and (adjF.chkIfOpinionWord(next3wordA)=="true")):#rule22 
                                                rule =" R22a "
                                                aspectStr1 = rule+"#"+wordA
                                                sentWord =wordA+" "+next3wordA
                                                prevArr = h-1
                                                if ((prevArr >= 0)and (prevArr< (len(wd1)))):
                                                    prevW =wd1[prevArr]
                                                    prevWordA = prevW[0]
                                                    prevWordB = prevW[1]
                                                    if ((prevWordB.startswith('N'))):
                                                        rule =" R22b "
                                                        aspectStr1 = rule+"#"+prevWordA+" "+wordA
                                                        sentWord =prevWordA+" "+wordA+" "+next3wordA
                                                    else:
                                                        rule =" R22c"
                                                        aspectStr1 = rule+"#"+wordA
                                                        sentWord =wordA+" "+next3wordA
                                                else:
                                                    rule =" R22d "
                                                    aspectStr1 = rule+"#"+wordA
                                                    sentWord =wordA+" "+next3wordA
                                    elif ((next2wordB.startswith('J')) and (adjF.chkIfOpinionWord(next2wordA)=="true")):#rule23
                                        rule =" R23a "
                                        aspectStr1 = rule+"#"+wordA
                                        prevArr = h-1
                                        if ((prevArr >= 0)and (prevArr< (len(wd1)))):
                                            prevW =wd1[prevArr]
                                            prevWordA = prevW[0]
                                            prevWordB = prevW[1]
                                            if ((prevWordB.startswith('N'))):
                                                rule =" R23b"
                                                aspectStr1 = rule+"#"+prevWordA+" "+wordA
                                                sentWord =prevWordA+" "+wordA+" "+next2wordA
                                            else:
                                                rule =" R23c "
                                                aspectStr1 = rule+"#"+wordA
                                                sentWord=wordA+" "+next2wordA
                                        else:
                                            rule =" R23d "
                                            aspectStr1 = rule+"#"+wordA
                                            sentWord =wordA+" "+next2wordA
                                    elif ((next2wordB.startswith('IN'))) :
                                        nextArr3 = h+3
                                        if (nextArr3<(len(wd1))):
                                            nextW3 =wd1[nextArr3]
                                            next3wordA = nextW3[0]
                                            next3wordB = nextW3[1]
                                            if ((next3wordB.startswith('N'))and (adjF.chkIfOpinionWord(next3wordA)=="true")):
                                                rule =" R27a "
                                                aspectStr1 = rule+"#"+wordA
                                                sentWord =wordA+" "+next3wordA
                                                prevArr = h-1
                                                if ((prevArr >= 0)and (prevArr< (len(wd1)))):
                                                    prevW =wd1[prevArr]
                                                    prevWordA = prevW[0]
                                                    prevWordB = prevW[1]
                                                    if ((prevWordB.startswith('N'))):
                                                        rule =" R27b "
                                                        aspectStr1 = rule+"#"+prevWordA+" "+wordA
                                                        sentWord =prevWordA+" "+wordA+" "+next3wordA
                                                    else:
                                                        rule =" R27c"
                                                        aspectStr1 = rule+"#"+wordA
                                                        sentWord =wordA+" "+next3wordA
                                                else:
                                                    rule =" R27d "
                                                    aspectStr1 = rule+"#"+wordA
                                                    sentWord =wordA+" "+next3wordA
                                            elif ((next3wordB.startswith('N'))and (adjF.chkIfOpinionWord(next3wordA)=="false")):
                                                rule =" R27e "
                                                aspectStr1 = rule+"#"+wordA   
                                                sentWord =wordA+" "+next3wordA
                                                prevArr = h-1
                                                if ((prevArr >= 0)and (prevArr< (len(wd1)))):
                                                    prevW =wd1[prevArr]
                                                    prevWordA = prevW[0]
                                                    prevWordB = prevW[1]
                                                    if ((prevWordB.startswith('N'))):
                                                        rule =" R27f "
                                                        aspectStr1 = rule+"#"+prevWordA+" "+wordA
                                                        sentWord =prevWordA+" "+wordA+" "+next3wordA
                                                    else:
                                                        rule =" R27g "
                                                        aspectStr1 = rule+"#"+wordA
                                                        sentWord =wordA+" "+next3wordA
                                                else:
                                                    rule =" R27h "
                                                    aspectStr1 = rule+"#"+wordA
                                                    sentWord =wordA+" "+next3wordA
                                    elif (adjF.chkIfOpinionWord(nextwordA)=="true"):#Rule 28 N -V combo    
                                        rule =" R28a "
                                        aspectStr1 = rule+"#"+wordA
                                        sentWord =wordA+" "+nextwordA
                                        prevArr = h-1
                                        if ((prevArr >= 0)and (prevArr< (len(wd1)))):
                                            prevW =wd1[prevArr]
                                            prevWordA = prevW[0]
                                            prevWordB = prevW[1]
                                            if ((prevWordB.startswith('N'))):
                                                rule =" R28b "
                                                aspectStr1 = rule+"#"+prevWordA+" "+wordA
                                                sentWord =wordA+" "+nextwordA
                                            else:
                                                rule =" R28c "
                                                aspectStr1 = rule+"#"+wordA
                                                sentWord =wordA+" "+nextwordA
                                        else:
                                            rule =" R28d "
                                            aspectStr1 = rule+"#"+wordA
                                            sentWord =wordA+" "+nextwordA
                                    elif  ((next2wordB.startswith('V'))) :
                                        
                                        if(adjF.chkIfOpinionWord(next2wordA)=="true"):
                                            rule =" R71a "
                                            aspectStr1 = rule+"#"+wordA  
                                            sentWord =wordA+" "+next2wordA
                                        nextArr3 = h+3
                                        if (nextArr3<(len(wd1))):
                                            nextW3 =wd1[nextArr3]
                                            next3wordA = nextW3[0]
                                            next3wordB = nextW3[1]
                                            if ((next3wordB.startswith('J'))and (adjF.chkIfOpinionWord(next3wordA)=="true")):
                                                rule =" R60a "
                                                aspectStr1 = rule+"#"+wordA 
                                                sentWord =wordA+" "+next3wordA
                                                prevArr = h-1
                                                if ((prevArr >= 0)and (prevArr< (len(wd1)))):
                                                    prevW =wd1[prevArr]
                                                    prevWordA = prevW[0]
                                                    prevWordB = prevW[1]
                                                    if ((prevWordB.startswith('N'))):
                                                        rule =" R60b "
                                                        aspectStr1 = rule+"#"+prevWordA+" "+wordA
                                                        sentWord =prevWordA+" "+wordA+" "+next3wordA
                                                    else:
                                                        rule =" R60c "
                                                        aspectStr1 = rule+"#"+wordA
                                                        sentWord =wordA+" "+next3wordA
                                                else:
                                                    rule =" R60d "
                                                    aspectStr1 = rule+"#"+wordA
                                                    sentWord =wordA+" "+next3wordA
                                    elif  ((next2wordB.startswith('N'))) :
                                        
                                        if(adjF.chkIfOpinionWord(next2wordA)=="true"):
                                            rule =" R72a "
                                            aspectStr1 = rule+"#"+wordA  
                                            sentWord =wordA+" "+next2wordA
                                elif (adjF.chkIfOpinionWord(nextwordA)=="true"):#Rule 28 N -V combo     
                                    rule =" R28a "
                                    aspectStr1 = rule+"#"+wordA
                                                               
                                    prevArr = h-1
                                    if ((prevArr >= 0)and (prevArr< (len(wd1)))):
                                        prevW =wd1[prevArr]
                                        prevWordA = prevW[0]
                                        prevWordB = prevW[1]
                                        if ((prevWordB.startswith('N'))):
                                            rule =" R28b "
                                            aspectStr1 = rule+"#"+prevWordA+" "+wordA
                                            sentWord =prevWordA+" "+wordA+" "+nextwordA
                                        else:
                                            rule =" R28c "
                                            aspectStr1 = rule+"#"+wordA
                                            sentWord =wordA+" "+nextwordA
                                    else:
                                        rule =" R28d "
                                        aspectStr1 = rule+"#"+wordA
                                        sentWord =wordA+" "+nextwordA
                    elif(((nextwordB.startswith('IN')))) :              
                        nextArr2 = h+2
                        if (nextArr2<(len(wd1))):
                            nextW2 =wd1[nextArr2]
                            next2wordA = nextW2[0]
                            next2wordB = nextW2[1]
                            if ((next2wordB.startswith('N'))): 
                                if (adjF.chkIfOpinionWord(next2wordA)=="true"):#rule14
                                    rule =" R14a "
                                    aspectStr1 = rule+"#"+wordA
                                    sentWord =prevWordA+" "+wordA+" "+next2wordA
                                    prevArr = h-1
                                    if ((prevArr >= 0)and (prevArr< (len(wd1)))):
                                        prevW =wd1[prevArr]
                                        prevWordA = prevW[0]
                                        prevWordB = prevW[1]
                                        if ((prevWordB.startswith('N'))):
                                            rule =" R14b "
                                            aspectStr1 = rule+"#"+prevWordA+" "+wordA
                                            sentWord =prevWordA+" "+wordA+" "+next2wordA
                                    else:
                                        rule =" R14d "
                                        aspectStr1 = rule+"#"+wordA
                                        sentWord =wordA+" "+next2wordA
                                else:
                                    nextArr3 = h+3
                                    if (nextArr3<(len(wd1))):
                                        nextW3 =wd1[nextArr3]
                                        next3wordA = nextW3[0]
                                        next3wordB = nextW3[1]
                                        if ((next3wordB.startswith('N'))):
                                            nextArr4 = h+4
                                            if (nextArr4<(len(wd1))):
                                                nextW4 =wd1[nextArr4]
                                                next4wordA = nextW4[0]
                                                next4wordB = nextW4[1]    
                                                if ((next4wordB.startswith('V'))):
                                                    nextArr5= h+5
                                                    if (nextArr5<(len(wd1))):
                                                        nextW5 =wd1[nextArr5]
                                                        next5wordA = nextW5[0]
                                                        next5wordB = nextW5[1]   
                                                        if ((next5wordB.startswith('DT')) or (next5wordB.startswith('N'))):
                                                            nextArr6= h+6
                                                            if (nextArr6<(len(wd1))):
                                                                nextW6 =wd1[nextArr6]
                                                                next6wordA = nextW6[0]
                                                                next6wordB = nextW6[1]  
                                                                if ((next6wordB.startswith('NN'))):
                                                                    nextArr7= h+7
                                                                    if (nextArr7<(len(wd1))):
                                                                        nextW7 =wd1[nextArr7]
                                                                        next7wordA = nextW7[0]
                                                                        next7wordB = nextW7[1]  
                                                        
                                                                        if( ((next7wordB.startswith('J')) or (next7wordB.startswith('V'))) and (adjF.chkIfOpinionWord(next7wordA)=="true")):#rule31c  )
                                                                            rule =" R31a "
                                                                            aspectStr1 = rule+"#"+wordA
                                                                            sentWord =prevWordA+" "+wordA+" "+next7wordA
                                                                            prevArr = h-1
                                                                            if ((prevArr >= 0)and (prevArr< (len(wd1)))):
                                                                                prevW =wd1[prevArr]
                                                                                prevWordA = prevW[0]
                                                                                prevWordB = prevW[1]
                                                                                if ((prevWordB.startswith('N'))):
                                                                                    rule =" R31b "
                                                                                    aspectStr1 = rule+"#"+prevWordA+" "+wordA
                                                                                    sentWord =prevWordA+" "+wordA+" "+next7wordA
                                                                                else:
                                                                                    rule =" R31c "
                                                                                    aspectStr1 = rule+"#"+wordA
                                                                                    sentWord =wordA+" "+next7wordA
                                                                            else: 
                                                                                rule =" R31d "
                                                                                aspectStr1 = rule+"#"+wordA
                                                                                sentWord =wordA+" "+next7wordA
                            elif((next2wordB.startswith('DT'))):
                                nextArr3 = h+3
                                if (nextArr3<(len(wd1))):
                                    nextW3 =wd1[nextArr3]
                                    next3wordA = nextW3[0]
                                    next3wordB = nextW3[1]
                                    if ((next3wordB.startswith('N'))):
                                        nextArr4 = h+4
                                        if (nextArr4<(len(wd1))):
                                            nextW4 =wd1[nextArr4]
                                            next4wordA = nextW4[0]
                                            next4wordB = nextW4[1]    
                                            if ((next4wordB.startswith('V'))):
                                                nextArr5= h+5
                                                if (nextArr5<(len(wd1))):
                                                    nextW5 =wd1[nextArr5]
                                                    next5wordA = nextW5[0]
                                                    next5wordB = nextW5[1]   
                                                    if ((next5wordB.startswith('J')) and (adjF.chkIfOpinionWord(next5wordA)=="true")):#rule17  )
                                                        rule =" R17a "
                                                        aspectStr1 = rule+"#"+wordA
                                                        sentWord =wordA+" "+next5wordA
                                                        prevArr = h-1
                                                        if ((prevArr >= 0)and (prevArr< (len(wd1)))):
                                                            prevW =wd1[prevArr]
                                                            prevWordA = prevW[0]
                                                            prevWordB = prevW[1]
                                                            if ((prevWordB.startswith('N'))):
                                                                rule =" R17b "
                                                                aspectStr1 = rule+"#"+prevWordA+" "+wordA
                                                                sentWord =prevWordA+" "+wordA+" "+next5wordA
                                                            else:
                                                                rule =" R17c "
                                                                aspectStr1 = rule+"#"+wordA
                                                                sentWord =wordA+" "+next5wordA
                                                        else: 
                                                            rule =" R17d "
                                                            aspectStr1 = rule+"#"+wordA
                                                            sentWord =wordA+" "+next5wordA
                                                    elif (next5wordB.startswith('IN')or (next5wordB.startswith('RP'))):
                                                        nextArr6= h+6
                                                        if (nextArr6<(len(wd1))):
                                                            nextW6 =wd1[nextArr6]
                                                            next6wordA = nextW6[0]
                                                            next6wordB = nextW6[1]   
                                                            if (next6wordB.startswith('N')):
                                                                nextArr7= h+7
                                                                if (nextArr7<(len(wd1))):
                                                                    nextW7 =wd1[nextArr7]
                                                                    next7wordA = nextW7[0]
                                                                    next7wordB = nextW7[1]
                                                                    if (next7wordB.startswith('R')): 
                                                                        nextArr8= h+8
                                                                        if (nextArr8<(len(wd1))):
                                                                            nextW8 =wd1[nextArr8]
                                                                            next8wordA = nextW8[0]
                                                                            next8wordB = nextW8[1]   
                                                                            if (next8wordB.startswith('R')): 
                                                                                if (adjF.chkIfOpinionWord(next8wordA)=="true"): #Rule19
                                                                                    rule =" R19a "
                                                                                    aspectStr1 = rule+"#"+wordA 
                                                                                    
                                                                                    sentWord =wordA+" "+next8wordA
                                                            
                                                                                    prevArr = h-1
                                                                                    if ((prevArr >= 0)and (prevArr< (len(wd1)))):
                                                                                        prevW =wd1[prevArr]
                                                                                        prevWordA = prevW[0]
                                                                                        prevWordB = prevW[1]
                                                                                        if ((prevWordB.startswith('N'))):
                                                                                            rule =" R19b "
                                                                                            aspectStr1 = rule+"#"+prevWordA+" "+wordA
                                                                                            sentWord =prevWordA+" "+wordA+" "+next8wordA
                                                                                        else:
                                                                                            rule =" R19c "
                                                                                            aspectStr1 = rule+"#"+wordA
                                                                                            sentWord =wordA+" "+next8wordA
                                                                                    else: 
                                                                                        rule =" R19d "
                                                                                        aspectStr1 = rule+"#"+wordA
                                                                                        sentWord =wordA+" "+next8wordA
                                                                                else:
                                                                                    nextArr9= h+9
                                                                                    if (nextArr9<(len(wd1))):
                                                                                        nextW9 =wd1[nextArr9]
                                                                                        next9wordA = nextW9[0]
                                                                                        next9wordB = nextW9[1]
                                                                                        if ((next9wordB.startswith('J')) and (adjF.chkIfOpinionWord(next9wordA)=="true")):#rule18  )
                                                                                            rule =" R18a "
                                                                                            aspectStr1 = rule+"#"+wordA
                                                                                            sentWord =wordA+" "+next9wordA
                                                                                            sentWord =wordA+" "+next9wordA
                                                                                            prevArr = h-1
                                                                                            if ((prevArr >= 0)and (prevArr< (len(wd1)))):
                                                                                                prevW =wd1[prevArr]
                                                                                                prevWordA = prevW[0]
                                                                                                prevWordB = prevW[1]
                                                                                                if ((prevWordB.startswith('N'))):
                                                                                                    rule =" R18b "
                                                                                                    aspectStr1 = rule+"#"+prevWordA+" "+wordA
                                                                                                    sentWord =prevWordA+" "+wordA+" "+next9wordA
                                                                                                else:
                                                                                                    rule =" R18c "
                                                                                                    aspectStr1 = rule+"#"+wordA
                                                                                                    sentWord =wordA+" "+next9wordA
                                                                                            else: 
                                                                                                rule =" R18d "
                                                                                                aspectStr1 = rule+"#"+wordA
                                                                                                sentWord =+wordA+" "+next9wordA
                                                                    if (((next7wordB.startswith('J')) or (next7wordB.startswith('J')))  and (adjF.chkIfOpinionWord(next7wordA)=="true")): #Rule30
                                                                        rule =" R31a "
                                                                        aspectStr1 = rule+"#"+wordA 
                                                                        sentWord =prevWordA+" "+wordA+" "+next7wordA
                                                                        prevArr = h-1
                                                                        if ((prevArr >= 0)and (prevArr< (len(wd1)))):
                                                                            prevW =wd1[prevArr]
                                                                            prevWordA = prevW[0]
                                                                            prevWordB = prevW[1]
                                                                            if ((prevWordB.startswith('N'))):
                                                                                rule =" R31b "
                                                                                aspectStr1 = rule+"#"+prevWordA+" "+wordA
                                                                                sentWord =prevWordA+" "+wordA+" "+next7wordA
                                                                            else:
                                                                                rule =" R31c "
                                                                                aspectStr1 = rule+"#"+wordA   
                                                                                sentWord =wordA+" "+next7wordA
                                                                        else:
                                                                            rule =" R31d "
                                                                            aspectStr1 = rule+"#"+wordA
                                                                            sentWord =wordA+" "+next7wordA
                                                            elif (next6wordB.startswith('DT') or (next6wordB.startswith('N'))):#Rule 31
                                                                nextArr7= h+7
                                                                if (nextArr7<(len(wd1))):
                                                                    nextW7 =wd1[nextArr7]
                                                                    next7wordA = nextW7[0]
                                                                    next7wordB = nextW7[1]   
                                                                    if (next7wordB.startswith('N')):
                                                                        nextArr8= h+8
                                                                        if (nextArr8<(len(wd1))):
                                                                            nextW8 =wd1[nextArr8]
                                                                            next8wordA = nextW8[0]
                                                                            next8wordB = nextW8[1]
                                                                            if (((next8wordB.startswith('J')) or (next8wordB.startswith('J')))  and (adjF.chkIfOpinionWord(next8wordA)=="true")): #Rule30
                                                                                rule =" R31e "
                                                                                aspectStr1 = rule+"#"+wordA  
                                                                                sentWord =wordA+" "+next8wordA
                                                                                prevArr = h-1
                                                                                if ((prevArr >= 0)and (prevArr< (len(wd1)))):
                                                                                    prevW =wd1[prevArr]
                                                                                    prevWordA = prevW[0]
                                                                                    prevWordB = prevW[1]
                                                                                    if ((prevWordB.startswith('N'))):
                                                                                        rule =" R31f "
                                                                                        aspectStr1 = rule+"#"+prevWordA+" "+wordA
                                                                                        sentWord =prevWordA+" "+wordA+" "+next8wordA
                                                                                    else:
                                                                                        rule =" R31g "
                                                                                        aspectStr1 = rule+"#"+wordA
                                                                                        sentWord =wordA+" "+next8wordA
                                                                                else:
                                                                                    rule =" R31h "
                                                                                    aspectStr1 = rule+"#"+wordA  
                                                                                    sentWord =wordA+" "+next8wordA
                                                    elif (next5wordB.startswith('N')):#Rule 30
                                                        nextArr6= h+6
                                                        if (nextArr6<(len(wd1))):
                                                            nextW6 =wd1[nextArr6]
                                                            next6wordA = nextW6[0]
                                                            next6wordB = nextW6[1]   
                                                            if (((next6wordB.startswith('J')) or (next6wordB.startswith('J')))  and (adjF.chkIfOpinionWord(next6wordA)=="true")): #Rule30
                                                                rule =" R30a "
                                                                aspectStr1 = rule+"#"+wordA
                                                                sentWord =wordA+" "+next6wordA
                                                                prevArr = h-1
                                                                if ((prevArr >= 0)and (prevArr< (len(wd1)))):
                                                                    prevW =wd1[prevArr]
                                                                    prevWordA = prevW[0]
                                                                    prevWordB = prevW[1]
                                                                    if ((prevWordB.startswith('N'))):
                                                                        rule =" R30b "
                                                                        aspectStr1 = rule+"#"+prevWordA+" "+wordA
                                                                        sentWord =prevWordA+" "+wordA+" "+next6wordA
                                                                    else:
                                                                        rule =" R30c "
                                                                        aspectStr1 = rule+"#"+wordA   
                                                                        sentWord =wordA+" "+next6wordA
                                                                else:
                                                                    rule =" R30d "
                                                                    aspectStr1 = rule+"#"+wordA  
                                                                    sentWord =wordA+" "+next6wordA
                                                    elif (next5wordB.startswith('RB')):#Rule 20
                                                        nextArr6= h+6
                                                        if (nextArr6<(len(wd1))):
                                                            nextW6 =wd1[nextArr6]
                                                            next6wordA = nextW6[0]
                                                            next6wordB = nextW6[1]   
                                                            if (next6wordB.startswith('IN')):
                                                                nextArr7= h+7
                                                                if (nextArr7<(len(wd1))):
                                                                    nextW7 =wd1[nextArr7]
                                                                    next7wordA = nextW7[0]
                                                                    next7wordB = nextW7[1]
                                                                    if (((next7wordB.startswith('N'))or (next7wordB.startswith('J'))) and (adjF.chkIfOpinionWord(next7wordA)=="true")): #Rule20
                                                                        rule =" R20a "
                                                                        aspectStr1 = rule+"#"+wordA  
                                                                        sentWord =wordA+" "+next7wordA
                                                                        prevArr = h-1
                                                                        if ((prevArr >= 0)and (prevArr< (len(wd1)))):
                                                                            prevW =wd1[prevArr]
                                                                            prevWordA = prevW[0]
                                                                            prevWordB = prevW[1]
                                                                            if ((prevWordB.startswith('N'))):
                                                                                rule =" R20b "
                                                                                aspectStr1 = rule+"#"+prevWordA+" "+wordA
                                                                                sentWord =prevWordA+" "+wordA+" "+next7wordA
                                                                            else:
                                                                                rule =" R20c "
                                                                                aspectStr1 = rule+"#"+wordA 
                                                                                sentWord =wordA+" "+next7wordA
                                                                        else:
                                                                            rule =" R20d"
                                                                            aspectStr1 = rule+"#"+wordA   
                                                                            sentWord =wordA+" "+next7wordA
                                                            elif (((next6wordB.startswith('N'))or (next6wordB.startswith('J'))) and (adjF.chkIfOpinionWord(next6wordA)=="true")): #Rule21
                                                                rule =" R21a "
                                                                aspectStr1 = rule+"#"+wordA 
                                                                sentWord =wordA+" "+next6wordA
                                                                prevArr = h-1
                                                                if ((prevArr >= 0)and (prevArr< (len(wd1)))):
                                                                    prevW =wd1[prevArr]
                                                                    prevWordA = prevW[0]
                                                                    prevWordB = prevW[1]
                                                                    if ((prevWordB.startswith('N'))):
                                                                        rule =" R21b "
                                                                        aspectStr1 = rule+"#"+prevWordA+" "+wordA
                                                                        sentWord =prevWordA+" "+wordA+" "+next6wordA
                                                                    else:
                                                                        rule =" R21c "
                                                                        aspectStr1 = rule+"#"+wordA  
                                                                        sentWord =wordA+" "+next6wordA
                                                                else:
                                                                    rule =" R21d "
                                                                    aspectStr1 = rule+"#"+wordA    
                                                                    sentWord =wordA+" "+next6wordA
                                                    elif (next5wordB.startswith('DT')):#Rule 30
                                                        nextArr6= h+6
                                                        if (nextArr6<(len(wd1))):
                                                            nextW6 =wd1[nextArr6]
                                                            next6wordA = nextW6[0]
                                                            next6wordB = nextW6[1]   
                                                            if (next6wordB.startswith('N')):
                                                                nextArr7= h+7
                                                                if (nextArr7<(len(wd1))):
                                                                    nextW7 =wd1[nextArr7]
                                                                    next7wordA = nextW7[0]
                                                                    next7wordB = nextW7[1]
                                                                    if ((next7wordB.startswith('J')) and (adjF.chkIfOpinionWord(next7wordA)=="true")): #Rule20
                                                                        rule =" R30a "
                                                                        aspectStr1 = rule+"#"+wordA   
                                                                        sentWord =wordA+" "+next7wordA
                                                                        prevArr = h-1
                                                                        if ((prevArr >= 0)and (prevArr< (len(wd1)))):
                                                                            prevW =wd1[prevArr]
                                                                            prevWordA = prevW[0]
                                                                            prevWordB = prevW[1]
                                                                            if ((prevWordB.startswith('N'))):
                                                                                rule =" R30b "
                                                                                aspectStr1 = rule+"#"+prevWordA+" "+wordA
                                                                                sentWord =prevWordA+" "+wordA+" "+next7wordA
                                                                            else:
                                                                                rule =" R30c "
                                                                                aspectStr1 = rule+"#"+wordA
                                                                                sentWord =wordA+" "+next7wordA
                                                                        else:
                                                                            rule =" R30d "
                                                                            aspectStr1 = rule+"#"+wordA  
                                                                            sentWord =wordA+" "+next7wordA
                                                            if (next6wordB.startswith('DT')):
                                                                nextArr7= h+7
                                                                if (nextArr7<(len(wd1))):
                                                                    nextW7 =wd1[nextArr7]
                                                                    next7wordA = nextW7[0]
                                                                    next7wordB = nextW7[1]
                                                                    if (((next7wordB.startswith('N'))or (next7wordB.startswith('J'))) and (adjF.chkIfOpinionWord(next7wordA)=="true")): #Rule68
                                                                        rule =" R68a "
                                                                        aspectStr1 = rule+"#"+wordA  
                                                                        sentWord =wordA+" "+next7wordA
                                                                        prevArr= h-1
                                                                        if ((prevArr >= 0)and (prevArr< (len(wd1)))):
                                                                            prevW =wd1[prevArr]
                                                                            prevWordA = prevW[0]
                                                                            prevWordB = prevW[1]
                                                                            if ((prevWordB.startswith('N'))):
                                                                                rule =" R68b "
                                                                                aspectStr1 = rule+"#"+prevWordA+" "+wordA
                                                                                sentWord =prevWordA+" "+wordA+" "+next7wordA
                                                                            else:
                                                                                rule =" R68c "
                                                                                aspectStr1 = rule+"#"+wordA 
                                                                                sentWord =wordA+" "+next7wordA
                                                                        else:
                                                                            rule =" R68d "
                                                                            aspectStr1 = rule+"#"+wordA    
                                                                            sentWord =wordA+" "+next7wordA
                                                    elif (next5wordB.startswith('N')):#Rule 30
                                                        nextArr6= h+6
                                                        if (nextArr6<(len(wd1))):
                                                            nextW6 =wd1[nextArr6]
                                                            next6wordA = nextW6[0]
                                                            next6wordB = nextW6[1]   
                                                            if ((next6wordB.startswith('J')) and (adjF.chkIfOpinionWord(next6wordA)=="true")): #Rule30
                                                                rule =" R30a"
                                                                aspectStr1 = rule+"#"+wordA
                                                                sentWord =wordA+" "+next6wordA
                                                                prevArr = h-1
                                                                if ((prevArr >= 0)and (prevArr< (len(wd1)))):
                                                                    prevW =wd1[prevArr]
                                                                    prevWordA = prevW[0]
                                                                    prevWordB = prevW[1]
                                                                    if ((prevWordB.startswith('N'))):
                                                                        rule =" R30b "
                                                                        aspectStr1 = rule+"#"+prevWordA+" "+wordA
                                                                        sentWord =prevWordA+" "+wordA+" "+next6wordA
                                                                    else:
                                                                        rule =" R30c "
                                                                        aspectStr1 = rule+"#"+wordA  
                                                                        sentWord =wordA+" "+next6wordA
                                                                else:
                                                                    rule =" R30d "
                                                                    aspectStr1 = rule+"#"+wordA    
                                                                    sentWord =wordA+" "+next6wordA
                                            else:
                                                rule =" R24a "
                                                aspectStr1 = rule+"#"+wordA
                                                sentWord =wordA+" "+next2wordA
                                                prevArr = h-1
                                                if ((prevArr >= 0)and (prevArr< (len(wd1)))):
                                                    prevW =wd1[prevArr]
                                                    prevWordA = prevW[0]
                                                    prevWordB = prevW[1]
                                                    if ((prevWordB.startswith('N'))):
                                                        rule =" R24b "
                                                        aspectStr1 = rule+"#"+prevWordA+" "+wordA
                                                        sentWord =prevWordA+" "+wordA+" "+next2wordA
                                                    else:
                                                        rule =" R24c "
                                                        aspectStr1 = rule+"#"+wordA
                                                        sentWord =wordA+" "+next2wordA
                                                else:
                                                        rule =" R24d "
                                                        aspectStr1 = rule+"#"+wordA
                                                        sentWord =wordA+" "+next2wordA
                                        else:
                                            rule =" R24e "
                                            aspectStr1 = rule+"#"+wordA
                                            sentWord =wordA+" "+next2wordA
                                            prevArr = h-1
                                            if ((prevArr >= 0)and (prevArr< (len(wd1)))):
                                                prevW =wd1[prevArr]
                                                prevWordA = prevW[0]
                                                prevWordB = prevW[1]
                                                if ((prevWordB.startswith('N'))):
                                                    rule =" R24f "
                                                    aspectStr1 = rule+"#"+prevWordA+" "+wordA
                                                    sentWord =prevWordA+" "+wordA+" "+next2wordA
                                                else:
                                                    rule =" R24g "
                                                    aspectStr1 = rule+"#"+wordA
                                                    sentWord =wordA+" "+next2wordA
                                            else:
                                                rule =" R24h "
                                                aspectStr1 = rule+"#"+wordA
                                                sentWord =wordA+" "+next2wordA
                    elif (nextwordB.startswith('N')) : 
                        
                        if((adjF.chkIfOpinionWord(nextwordA)=="true")):
                            rule =" R75a "
                            aspectStr1 = rule+"#"+wordA
                            sentWord =wordA+" "+nextwordA
                        nextArr2= h+2
                        if (nextArr2<(len(wd1))):
                            nextW2 =wd1[nextArr2]
                            next2wordA = nextW2[0]
                            next2wordB = nextW2[1]
                            if ((((next2wordB.startswith('J')) or (next2wordB.startswith('R'))) and (adjF.chkIfOpinionWord(next2wordA)=="true"))):                  
                                rule =" R15a R74a "
                                aspectStr1 = rule+"#"+wordA
                                sentWord =wordA+" "+next2wordA
                    elif ((nextwordB.startswith('J')) and (adjF.chkIfOpinionWord(nextwordA)=="true")):  
                        nextArr2= h+2
                        if (nextArr2<(len(wd1))):
                            nextW2 =wd1[nextArr2]
                            next2wordA = nextW2[0]
                            next2wordB = nextW2[1]
                            if (not(next2wordB.startswith('N'))):
                                rule =" R25a "
                                aspectStr1 = rule+"#"+wordA
                                sentWord =wordA+" "+next2wordA
                                prevArr = h-1
                                if ((prevArr >= 0)and (prevArr< (len(wd1)))):
                                    prevW =wd1[prevArr]
                                    prevWordA = prevW[0]
                                    prevWordB = prevW[1]
                                    if ((prevWordB.startswith('N'))):
                                        rule =" R25b "
                                        aspectStr1 = rule+"#"+prevWordA+" "+wordA
                                        sentWord =prevWordA+" "+wordA+" "+nextwordA
                                    else:
                                        rule =" R25c "
                                        aspectStr1 = rule+"#"+wordA
                                        sentWord =wordA+" "+nextwordA
                                else :
                                    rule =" R25d "
                                    aspectStr1 = rule+"#"+wordA
                                    sentWord =wordA+" "+nextwordA
                            else:
                                rule =" R26a R28a "
                                aspectStr1 = rule+"#"+wordA   
                                sentWord =wordA+" "+nextwordA
                                prevArr = h-1
                                if ((prevArr >= 0)and (prevArr< (len(wd1)))):
                                    prevW =wd1[prevArr]
                                    prevWordA = prevW[0]
                                    prevWordB = prevW[1]
                                    if ((prevWordB.startswith('N'))):
                                        rule =" R26b R28b "
                                        aspectStr1 = rule+"#"+prevWordA+" "+wordA
                                        sentWord =prevWordA+" "+wordA+" "+nextwordA
                                    else:
                                        rule =" R26c R28c "
                                        spectStr1 = rule+"#"+wordA
                                        sentWord =wordA+" "+nextwordA
                                else:
                                    rule =" R26d R28d "
                                    aspectStr1 = rule+"#"+wordA
                                    sentWord =wordA+" "+nextwordA
                        else:
                            rule =" R26e R28e "
                            aspectStr1 = rule+"#"+wordA    
                            sentWord =wordA+" "+nextwordA
                            prevArr = h-1
                            if ((prevArr >= 0)and (prevArr< (len(wd1)))):
                                prevW =wd1[prevArr]
                                prevWordA = prevW[0]
                                prevWordB = prevW[1]
                                if ((prevWordB.startswith('N'))):
                                    rule =" R26f R28f "
                                    aspectStr1 = rule+"#"+prevWordA+" "+wordA
                                    sentWord =prevWordA+" "+wordA+" "+nextwordA
                                else:
                                    rule =" R26g R28g "
                                    aspectStr1 = rule+"#"+wordA
                                    sentWord =wordA+" "+nextwordA
                                    
                            else:
                                rule =" R26e R28e "
                                aspectStr1 = rule+"#"+wordA    
                                sentWord =wordA+" "+nextwordA
            if ((wordB.startswith('V')) ) :#rule12      
                nextArr = h+1
                if (nextArr< (len(wd1))):
                    nextW =wd1[nextArr]
                    nextwordA = nextW[0]
                    nextwordB = nextW[1]
                    if (nextwordB.startswith('N')):
                        if ((adjF.chkIfOpinionWord(nextwordA)=="true")) :
                            nextArr2 = h+2
                            if (nextArr2< (len(wd1))):
                                nextW2 =wd1[nextArr2]
                                next2wordA = nextW2[0]
                                next2wordB = nextW2[1]
                                if (next2wordB.startswith('N')):
                                    rule =" R32a "
                                    aspectStr1 = rule+"#"+next2wordA
                                    sentWord =nextwordA+" "+next2wordA
                                elif(next2wordB.startswith('R')):
                                    rule =" R44a "
                                    aspectStr1 = rule+"#"+next2wordA
                                    sentWord =nextwordA+" "+next2wordA
                                else:
                                    rule =" R32b"
                                    aspectStr1 = rule+"#"+next2wordA
                                    sentWord =nextwordA+" "+next2wordA
                        elif ((adjF.chkIfOpinionWord(wordA)=="true")) :
                            rule =" R65a "
                            aspectStr1 = rule+"#"+nextwordA
                            sentWord =wordA+" "+nextwordA
                            nextArr2 = h+2
                            if (nextArr2< (len(wd1))):
                                nextW2 =wd1[nextArr2]
                                next2wordA = nextW2[0]
                                next2wordB = nextW2[1]
                                if (next2wordB.startswith('N')):
                                    rule =" R65b "
                                    aspectStr1 = rule+"#"+nextwordA+" "+next2wordA
                                    sentWord =wordA+" "+nextwordA+" "+next2wordA
                                else:
                                    rule =" R65c "
                                    aspectStr1 = rule+"#"+nextwordA
                                    sentWord =wordA+" "+nextwordA+" "+next2wordA
                            else:
                                rule =" R65d "
                                aspectStr1 = rule+"#"+nextwordA
                                sentWord =wordA+" "+nextwordA
                    elif (nextwordB.startswith('DT')):
                        nextArr2 = h+2
                        if (nextArr2< (len(wd1))):
                            nextW2 =wd1[nextArr2]
                            next2wordA = nextW2[0]
                            next2wordB = nextW2[1]
                            if ((next2wordB.startswith('N'))):
                                if (adjF.chkIfOpinionWord(wordA)=="true"):
                                    rule =" R33b"
                                    aspectStr1 = rule+"#"+next2wordA
                                    sentWord =wordA+" "+next2wordA
                                    nextArr3 = h+3
                                    if (nextArr3<(len(wd1))):
                                        nextW3 =wd1[nextArr3]
                                        next3wordA = nextW3[0]
                                        next3wordB = nextW3[1]
                                        if ((next3wordB.startswith('N'))):
                                            rule =" R33c "
                                            aspectStr1 = rule+"#"+next2wordA+" "+next3wordA
                                            sentWord =wordA+" "+next2wordA+" "+next3wordA
                                        else:
                                            rule =" R33d "
                                            aspectStr1 = rule+"#"+next2wordA
                                            sentWord =wordA+" "+next2wordA
                                    else:
                                        rule =" R33e "
                                        aspectStr1 = rule+"#"+next2wordA
                                        sentWord =wordA+" "+next2wordA
                    elif (nextwordB.startswith('IN')):
                        nextArr2 = h+2
                        if (nextArr2< (len(wd1))):
                            nextW2 =wd1[nextArr2]
                            next2wordA = nextW2[0]
                            next2wordB = nextW2[1]
                            if (next2wordB.startswith('DT')):
                                nextArr3 = h+3
                                if (nextArr3<(len(wd1))):
                                    nextW3 =wd1[nextArr3]
                                    next3wordA = nextW3[0]
                                    next3wordB = nextW3[1]
                                    if (next3wordB.startswith('IN')):
                                        nextArr4 = h+4
                                        if (nextArr4<(len(wd1))):
                                            nextW4 =wd1[nextArr4]
                                            next4wordA = nextW4[0]
                                            next4wordB = nextW4[1]
                                            if ((next4wordB.startswith('N')) and (adjF.chkIfOpinionWord(next4wordA)=="true")):
                                                rule =" R35a "
                                                aspectStr1 = rule+"#"+wordA
                                                sentWord =wordA+" "+next4wordA
                                    elif ((next3wordB.startswith('N')) and (adjF.chkIfOpinionWord(next3wordA)=="true")):
                                        rule =" R35b R50b "
                                        aspectStr1 = rule+"#"+wordA
                                        sentWord =wordA+" "+next3wordA
                            if ((next2wordB.startswith('N')) and (adjF.chkIfOpinionWord(next2wordA)=="true")):
                                rule =" R35c "
                                aspectStr1 = rule+"#"+wordA
                                sentWord =wordA+" "+next2wordA
                    elif ((nextwordB.startswith('R'))and (adjF.chkIfOpinionWord(nextwordA)=="true")):
                        next2Arr = h+2
                        if (next2Arr< (len(wd1))):
                            nextW2 =wd1[next2Arr]
                            next2wordA = nextW2[0]
                            next2wordB = nextW2[1]
                            if (next2wordB.startswith('J')and (adjF.chkIfOpinionWord(next2wordA)=="false")):
                                rule =" R47a "
                                aspectStr1 = rule+"#"+next2wordA
                                sentWord =wordA+" "+next2wordA
                    elif (nextwordB.startswith('V')):
                        next2Arr = h+2
                        if (next2Arr< (len(wd1))):
                            nextW2 =wd1[next2Arr]
                            next2wordA = nextW2[0]
                            next2wordB = nextW2[1]
                            if (next2wordB.startswith('DT')):
                                next3Arr = h+3
                                if (next3Arr< (len(wd1))):
                                    nextW3 =wd1[next3Arr]
                                    next3wordA = nextW3[0]  
                                    next3wordB = nextW3[1]   
                                    if (next3wordB.startswith('V')and (adjF.chkIfOpinionWord(next3wordA)=="true")):
                                        rule =" R43a "
                                        aspectStr1 = rule+"#"+wordA
                                        sentWord =wordA+" "+next3wordA
                if (adjF.chkIfOpinionWord(wordA)=="true"):
                    nextArr = h+1
                    if (nextArr< (len(wd1))):
                        nextW =wd1[nextArr]
                        nextwordA = nextW[0]
                        nextwordB = nextW[1]
                        if((nextwordB.startswith('V'))):
                            nextArr2 = h+2
                            if (nextArr2<(len(wd1))):
                                        nextW2 =wd1[nextArr2]
                                        next2wordA = nextW2[0]
                                        next2wordB = nextW2[1]
                                        if ((next2wordB.startswith('N'))): #Rule 9
                                            rule =" R62a "
                                            aspectStr1 = rule+"#"+next2wordA
                                            sentWord =wordA+" "+next2wordA
                                            nextArr3 = h+3
                                            if (nextArr3<(len(wd1))):
                                                nextW3 =wd1[nextArr3]
                                                next3wordA = nextW3[0]
                                                next3wordB = nextW3[1]
                                                if ((next3wordB.startswith('C'))):
                                                    nextArr4 = h+4
                                                    if (nextArr4<(len(wd1))):
                                                        nextW4 =wd1[nextArr4]
                                                        next4wordA = nextW4[0]
                                                        next4wordB = nextW4[1]
                                                        if ((next4wordB.startswith('N'))):
                                                            rule =" R62b "
                                                            aspectStr1 = rule+"#"+next2wordA+":"+rule+"#"+next4wordA
                                                            sentWord =wordA+" "+next2wordA+":"+wordA+" "+next4wordA
                                                            nextArr5 = h+5
                                                            if (nextArr5<(len(wd1))):
                                                                nextW5 =wd1[nextArr5]
                                                                next5wordA = nextW5[0]
                                                                next5wordB = nextW5[1]
                                                                if ((next5wordB.startswith('N'))):
                                                                    rule =" R62e "
                                                                    aspectStr1 = rule+"#"+next2wordA+":"+rule+"#"+next4wordA+" "+next5wordA
                                                                    sentWord =wordA+" "+next2wordA+":"+wordA+" "+next4wordA+" "+next5wordA
                                                                else :
                                                                    rule =" R62f "
                                                                    aspectStr1 = rule+"#"+next2wordA+":"+rule+"#"+next4wordA
                                                                    sentWord =wordA+" "+next2wordA+":"+wordA+" "+next4wordA
                                                            else :
                                                                rule =" R62g "
                                                                aspectStr1 = rule+"#"+next2wordA+":"+rule+"#"+next4wordA
                                                                
                                                                sentWord =wordA+" "+next2wordA+":"+wordA+" "+next4wordA
                                                        else :
                                                            rule =" R62h "
                                                            aspectStr1 = rule+"#"+next2wordA
                                                            
                                                            sentWord =wordA+" "+next2wordA
                                                            
                                                elif ((next3wordB.startswith('N'))):
                                                    rule =" R62j "
                                                    aspectStr1 = rule+"#"+next2wordA+" "+next3wordA           
                                                    sentWord =wordA+" "+next2wordA+" "+next3wordA  
                                                    nextArr4 = h+4
                                                    if (nextArr4<(len(wd1))):
                                                        nextW4 =wd1[nextArr4]
                                                        next4wordA = nextW4[0]
                                                        next4wordB = nextW4[1]
                                                        if ((next4wordB.startswith('N'))):
                                                            rule =" R62k "
                                                            aspectStr1 = rule+"#"+next2wordA+" "+next3wordA+" "+next4wordA
                                                            sentWord =wordA+" "+next2wordA+" "+next3wordA+" "+next4wordA
                                                        else :
                                                            rule =" R62l "
                                                            aspectStr1 = rule+"#"+next2wordA+" "+next3wordA
                                                            sentWord =wordA+" "+next2wordA+" "+next3wordA
                                                    else :
                                                        rule =" R62m "
                                                        aspectStr1 = rule+"#"+next2wordA+" "+next3wordA
                                                        sentWord =wordA+" "+next2wordA+" "+next3wordA
                                                else:
                                                    rule =" R62n "
                                                    aspectStr1 = rule+"#"+next2wordA
                                                    sentWord =wordA+" "+next2wordA
                                            else:
                                                rule =" R62p "
                                                aspectStr1 = rule+"#"+next2wordA
                                                sentWord =wordA+" "+next2wordA
                                        if ((next2wordB.startswith('DT'))):
                                                nextArr3 = h+3
                                                if (nextArr3<(len(wd1))):
                                                    nextW3 =wd1[nextArr3]
                                                    next3wordA = nextW3[0]
                                                    next3wordB = nextW3[1]
                                                    if ((next3wordB.startswith('N'))):
                                                        rule =" R62q "
                                                        aspectStr1 = rule+"#"+next3wordA
                                                        sentWord =wordA+" "+next3wordA
                                                        nextArr4 = h+4
                                                        if (nextArr4<(len(wd1))):
                                                            nextW4 =wd1[nextArr4]
                                                            next4wordA = nextW4[0]
                                                            next4wordB = nextW4[1]
                                                            if ((next4wordB.startswith('CC'))):
                                                                nextArr5 = h+5
                                                                if (nextArr5<(len(wd1))):
                                                                    nextW5 =wd1[nextArr5]
                                                                    next5wordA = nextW5[0]
                                                                    next5wordB = nextW5[1]
                                                                    if ((next5wordB.startswith('DT'))):
                                                                            nextArr6 = h+6
                                                                            if (nextArr6<(len(wd1))):
                                                                                nextW6 =wd1[nextArr6]
                                                                                next6wordA = nextW6[0]
                                                                                next6wordB = nextW6[1]
                                                                                if ((next6wordB.startswith('N'))):
                                                                                    rule =" R62r "
                                                                                    aspectStr1 = rule+"#"+next3wordA+":"+rule+"#"+next6wordA
                                                                                    sentWord =wordA+" "+next3wordA+":"+wordA+" "+next6wordA
                                                                                    nextArr7 = h+7
                                                                                    if (nextArr7<(len(wd1))):
                                                                                        nextW7 =wd1[nextArr7]
                                                                                        next7wordA = nextW7[0]
                                                                                        next7wordB = nextW7[1]
                                                                                        if ((next7wordB.startswith('CC'))):
                                                                                            nextArr8 = h+8
                                                                                            if (nextArr8<(len(wd1))):
                                                                                                nextW8 =wd1[nextArr8]
                                                                                                next8wordA = nextW8[0]
                                                                                                next8wordB = nextW8[1]
                                                                                                if ((next8wordB.startswith('DT'))):
                                                                                                    nextArr9 = h+9
                                                                                                    if (nextArr9<(len(wd1))):
                                                                                                        nextW9 =wd1[nextArr9]
                                                                                                        next9wordA = nextW9[0]
                                                                                                        next9wordB = nextW9[1]
                                                                                                        if ((next9wordB.startswith('N'))):
                                                                                                            rule =" R62s "
                                                                                                            aspectStr1 = rule+"#"+next3wordA+":"+rule+"#"+next6wordA+":"+rule+"#"+next9wordA
                                                                                                            sentWord =wordA+" "+next3wordA+":"+wordA+" "+next6wordA+":"+wordA+" "+next9wordA
                                                                                                        else:
                                                                                                            rule =" R62t "
                                                                                                            aspectStr1 = rule+"#"+next3wordA+":"+rule+"#"+next6wordA+":"+rule+"#"+next9wordA
                                                                                                            sentWord =wordA+" "+next3wordA+":"+wordA+" "+next6wordA+":"+wordA+" "+next9wordA
                                                                                                    else:
                                                                                                        rule =" R62u "
                                                                                                        aspectStr1 = rule+"#"+next3wordA+":"+rule+"#"+next6wordA
                                                                                                        sentWord =wordA+" "+next3wordA+":"+wordA+" "+next6wordA
                                                                                                elif ((next8wordB.startswith('N'))):
                                                                                                        rule =" R62v "
                                                                                                        aspectStr1 = rule+"#"+next3wordA+":"+rule+"#"+next6wordA+":"+rule+"#"+next8wordA
                                                                                                        sentWord =wordA+" "+next3wordA+":"+wordA+" "+next6wordA+":"+wordA+" "+next8wordA
                                                                                                elif ((next8wordB.startswith('V'))):
                                                                                                        rule =" R62w "
                                                                                                        aspectStr1 = rule+"#"+next3wordA+":"+rule+"#"+next6wordA+":"+rule+"#"+next8wordA
                                                                                                        sentWord =wordA+" "+next3wordA+":"+wordA+" "+next6wordA+":"+wordA+" "+next8wordA
                                                                                                else:
                                                                                                    rule =" R62wa "
                                                                                                    aspectStr1 = rule+"#"+next3wordA+":"+rule+"#"+next6wordA+":"+rule+"#"+next8wordA
                                                                                                    sentWord =wordA+" "+next3wordA+":"+wordA+" "+next6wordA+":"+wordA+" "+next8wordA
                                                                                            else:
                                                                                                rule =" R62aa "
                                                                                                aspectStr1 = rule+"#"+next3wordA+":"+rule+"#"+next6wordA
                                                                                                
                                                                                                sentWord =wordA+" "+next3wordA+":"+wordA+" "+next6wordA
                                                                                        else:
                                                                                                rule =" R62ab "
                                                                                                aspectStr1 = rule+"#"+next3wordA+":"+rule+"#"+next6wordA

                                                                                                sentWord =wordA+" "+next3wordA+":"+wordA+" "+next6wordA
                                                                                                
                                                                                    else:
                                                                                        rule =" R62ac "
                                                                                        aspectStr1 = rule+"#"+next3wordA+":"+rule+"#"+next6wordA
                                                                                        
                                                                                        sentWord =wordA+" "+next3wordA+":"+wordA+" "+next6wordA
                                                                                else:
                                                                                    rule =" R62ad "
                                                                                    aspectStr1 = rule+"#"+next3wordA+":"+rule+"#"+next6wordA
                                                                                    
                                                                                    sentWord =wordA+" "+next3wordA+":"+wordA+" "+next6wordA
                                                                            else:
                                                                                rule =" R62ae "
                                                                                aspectStr1 = rule+"#"+next3wordA
                                                                                
                                                                                sentWord =wordA+" "+next3wordA
                                                                    else:
                                                                        rule =" R62af "
                                                                        aspectStr1 = rule+"#"+next3wordA
                                                                        
                                                                        sentWord =wordA+" "+next3wordA
                                                                else:
                                                                    rule =" R62ag "
                                                                    aspectStr1 = rule+"#"+next3wordA
                                                                    sentWord =wordA+" "+next3wordA
                                                            elif (next4wordB.startswith('N')):
                                                                rule =" R62af "
                                                                aspectStr1 = rule+"#"+next3wordA +" "+next4wordA
                                                                sentWord =wordA+" "+next3wordA+" "+next4wordA
                                                            else:
                                                                rule =" R62ah "
                                                                aspectStr1 = rule+"#"+next3wordA
                                                                sentWord =wordA+" "+next3wordA
                                                        else:
                                                            rule =" R62aj "
                                                            aspectStr1 = rule+"#"+next3wordA
                                                            sentWord =wordA+" "+next3wordA
                                                    else:
                                                        rule =" R62ak "
                                                        aspectStr1 = rule+"#"+next3wordA
                                                        sentWord =wordA+" "+next3wordA
            if ((wordB.startswith('R')) ) :
                phase0=""
                phase1=""
                nextArr = h+1
                if (nextArr< (len(wd1))):
                    phase0 ="yes"
                    nextW =wd1[nextArr]
                    nextwordA = nextW[0]
                    nextwordB = nextW[1]
                    nextArr2 = h+2
                    if (nextArr2< (len(wd1))):
                        phse1="yes"
                        nextW2 =wd1[nextArr2]
                        next2wordA = nextW2[0]
                        next2wordB = nextW2[1]    
                    
                if (adjF.chkIfOpinionWord(wordA)=="true"):
                    if (phase0=="yes"):
                        a = ""
                    elif (nextwordB.startswith('R')):
                            rule =" R40a "
                            aspectStr1 = rule+"#"+nextwordA
                            sentWord =wordA+" "+nextwordA
                elif (adjF.chkIfOpinionWord(wordA)=="false"):
                    if (nextwordB.startswith('J')):
                        if (adjF.chkIfOpinionWord(nextwordA)=="true"):
                            if (phase1=="true") :
                                if (next2wordB.startswith('N')):
                                    rule =" R39a "
                                    aspectStr1 = rule+"#"+next2wordA
                                    sentWord =nextwordA+" "+next2wordA
                                if (not(next2wordB.startswith('N'))):#rule37
                                    rule =" R37b "
                                    aspectStr1 = rule+"#"+next2wordA 
                                    sentWord =nextwordA+" "+next2wordA
                                if (next2wordB.startswith('R')):
                                    rule =" R40b "
                                    aspectStr1 = rule+"#"+next2wordA
                                    sentWord =nextwordA+" "+next2wordA
                    elif (nextwordB.startswith('R')):    
                        if (phase1=="true") :
                            if (next2wordB.startswith('J')):
                                    rule =" R41a "
                                    aspectStr1 = rule+"#"+next2wordA
                                    sentWord =wordA+" "+next2wordA
            if ((wordB.startswith('IN'))  and (adjF.chkIfOpinionWord(wordA)=="true")):#rule63    
                nextArr = h+1
                if (nextArr< (len(wd1))):
                    nextW =wd1[nextArr]
                    nextwordA = nextW[0]
                    nextwordB = nextW[1]
                    if (nextwordB.startswith('DT')):
                        nextArr2 = h+2
                        if (nextArr2< (len(wd1))):
                            nextW2 =wd1[nextArr2]
                            next2wordA = nextW2[0]
                            next2wordB = nextW2[1]
                            if (next2wordB.startswith('N')):
                                rule =" R63r "
                                aspectStr1 = rule+"#"+next2wordA
                                
                                sentWord =wordA+" "+next2wordA
                            elif ((next2wordB.startswith('V') )or (next2wordB.startswith('J'))):
                                rule =" R63s "
                                aspectStr1 = rule+"#"+next2wordA
                                
                                sentWord =wordA+" "+next2wordA
                    elif ((next2wordB.startswith('V') )or (next2wordB.startswith('J')) or (next2wordB.startswith('N'))):
                        rule =" R63t "
                        aspectStr1 = rule+"#"+next2wordA
                        sentWord =wordA+" "+next2wordA
            if (((wordB.startswith('DT')) and (wordA=="no"))  and (adjF.chkIfOpinionWord(wordA)=="true")):#rule70   
        
                nextArr = h+1
                if (nextArr< (len(wd1))):
                    nextW =wd1[nextArr]
                    nextwordA = nextW[0]
                    nextwordB = nextW[1]
                    if (nextwordB.startswith('N')):
                        rule =" R70a "
                        aspectStr1 = rule+"#"+nextwordA
                     
                        sentWord =wordA+" "+nextwordA
            if ((len(aspectStr1)>2)):
                bStr =   aspectStr1.split(":")   
                for y in range (len(bStr)):
                    fW = bStr[y]
                    if (chkIfAlreadyExistInArray2(fW,fullPhrase)=="false"):
                        if (len(fullPhrase)>1):
                            fullPhrase = fullPhrase +"|" +fW 
                            
                        else :    
                            fullPhrase =fW
                           
                            
                            
                    if (chkIfAlreadyExistInArray2(sentWord, sentPhrase)=="false"):
                        if (len( sentPhrase)>1):
                            sentPhrase = sentPhrase +"|"+sentWord
                        else :    
                            sentPhrase =sentWord
                            
        if ((len( fullPhrase))>1):
           
            fullPhrase= reviewID+":"+ fullPhrase
            sentPhrase = reviewID+":"+ sentPhrase
            nounPhrases.append(  fullPhrase)
            sentPhrases.append(sentPhrase)
        else:
            fullPhrase= reviewID+":"
            sentPhrase = reviewID+":"
            nounPhrases.append( fullPhrase)
            sentPhrases.append(sentPhrase)
            
data=nounPhrases
data1=sentPhrases

df = pd.DataFrame(data,columns=['Id'])
np.savetxt(r'rulesOut.txt', df.values, fmt='%s',delimiter ='|')


df1 = pd.DataFrame(data1,columns=['Id'])
np.savetxt(r'sentOut.txt', df1.values, fmt='%s',delimiter ='|')